# 📊 Reporte de Pull Requests Merged en Bitbucket

Este reporte proporciona una vista completa de todos los pull requests que han sido exitosamente merged en los repositorios de Bitbucket, incluyendo información detallada sobre comentarios, tiempos de resolución y métricas de productividad para análisis histórico y mejora de procesos de desarrollo.

## 🔧 Configuración e Importación de Librerías

In [ ]:
%pip install python-dotenv requests pandas matplotlib seaborn

import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import threading
from concurrent.futures import ThreadPoolExecutor
import time

print("✅ Librerías importadas correctamente")

In [ ]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Configurar credenciales de Bitbucket
username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")

# Verificar que las credenciales estén configuradas
if not all([username, app_password, workspace]):
    print("❌ Error: Asegúrate de tener configuradas las variables de entorno:")
    print("- BITBUCKET_USERNAME")
    print("- BITBUCKET_APP_PASSWORD")
    print("- BITBUCKET_WORKSPACE")
else:
    print(f"✅ Credenciales configuradas correctamente para el workspace: {workspace}")
    print(f"   Usuario: {username}")

## 📦 Obtener Repositorios y Funciones de Utilidad

In [ ]:
# Función para obtener todos los elementos paginados de la API de Bitbucket
def get_all_items(url, params=None):
    """
    Obtiene todos los elementos de una URL paginada de la API de Bitbucket
    """
    items = []
    while url:
        response = requests.get(url, params=params, auth=(username, app_password))
        if response.status_code != 200:
            print(
                f"❌ Error al obtener datos: {response.status_code} - {response.text}"
            )
            break

        data = response.json()
        items.extend(data.get("values", []))

        # Obtener la siguiente página si existe
        url = data.get("next")
        params = None  # Los parámetros ya están incluidos en la URL 'next'

    return items


# Función para obtener comentarios de un pull request
def get_pr_comments(workspace, repo_slug, pr_id):
    """
    Obtiene todos los comentarios de un pull request específico
    """
    comments_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}/comments"
    return get_all_items(comments_url)


# Función para convertir fechas a formato "para humanos" en español
def fecha_para_humanos(fecha_str):
    """
    Convierte una fecha en formato datetime o string a texto legible en español
    """
    try:
        # Si ya es string formateado, intentar parsearlo
        if isinstance(fecha_str, str):
            # Intentar varios formatos posibles
            try:
                fecha = pd.to_datetime(fecha_str, format="%Y-%m-%d %H:%M")
            except:
                fecha = pd.to_datetime(fecha_str)
        else:
            # Si es un objeto datetime de pandas
            fecha = pd.to_datetime(fecha_str)

        # Asegurar que la fecha tenga zona horaria UTC para comparación
        if fecha.tz is None:
            fecha = fecha.tz_localize("UTC")

        # Obtener tiempo actual en UTC
        ahora = pd.Timestamp.now(tz="UTC")

        # Calcular diferencia
        diff = ahora - fecha

        # Obtener componentes
        dias = diff.days
        segundos_totales = diff.total_seconds()
        horas = int(segundos_totales // 3600)
        minutos = int((segundos_totales % 3600) // 60)

        if dias > 0:
            if dias == 1:
                return "hace 1 día"
            elif dias < 7:
                return f"hace {dias} días"
            elif dias < 30:
                semanas = dias // 7
                if semanas == 1:
                    return "hace 1 semana"
                else:
                    return f"hace {semanas} semanas"
            elif dias < 365:
                meses = dias // 30
                if meses == 1:
                    return "hace 1 mes"
                else:
                    return f"hace {meses} meses"
            else:
                años = dias // 365
                if años == 1:
                    return "hace 1 año"
                else:
                    return f"hace {años} años"
        elif horas > 0:
            if horas == 1:
                return "hace 1 hora"
            else:
                return f"hace {horas} horas"
        elif minutos > 0:
            if minutos == 1:
                return "hace 1 minuto"
            else:
                return f"hace {minutos} minutos"
        else:
            return "hace menos de 1 minuto"
    except Exception as e:
        print(f"Error procesando fecha {fecha_str}: {e}")
        return str(fecha_str)  # Si hay error, devolver la fecha original como string


# Función para analizar comentarios del pull request según las nuevas reglas
def analyze_pr_comments(comments, pr_author_name):
    """
    Analiza los comentarios de un PR y los clasifica según las nuevas reglas (mutuamente excluyentes):
    - Devoluciones: Comentarios normales de usuarios diferentes al autor del PR (NO inline)
    - Estandarizaciones_Codigo: Comentarios inline de usuarios diferentes al autor del PR
    """
    comm_pullrequest = 0
    estandarizaciones_codigo = 0

    for comment in comments:
        # Solo procesar comentarios no eliminados
        if comment.get("deleted", False):
            continue

        # Obtener el nombre del usuario que hizo el comentario
        comment_author = None
        if "user" in comment and comment["user"]:
            # Priorizar display_name, luego nickname
            comment_author = comment["user"].get("display_name") or comment["user"].get(
                "nickname"
            )

        # Solo contar si el comentario es de un usuario diferente al autor del PR
        if comment_author and comment_author != pr_author_name:
            # Verificar si es comentario inline (desarrollador) o normal (pull request)
            if "inline" in comment and comment["inline"]:
                # Es un comentario inline -> comentario de estandarización de código
                estandarizaciones_codigo += 1
            else:
                # Es un comentario normal -> comentario del pull request
                comm_pullrequest += 1

    return comm_pullrequest, estandarizaciones_codigo


# Función para acortar nombres largos
def acortar_nombre(nombre_completo):
    """
    Acorta nombres largos para mostrar solo primer nombre + primera letra del segundo nombre
    Ejemplo: "Daniel Felipe Leal Chaves" -> "DanielF."
    """
    if not nombre_completo or not isinstance(nombre_completo, str):
        return nombre_completo

    # Dividir el nombre en partes
    partes = nombre_completo.strip().split()

    if len(partes) == 1:
        # Solo tiene un nombre
        return partes[0]
    elif len(partes) >= 2:
        # Tiene al menos dos nombres
        primer_nombre = partes[0]
        segunda_parte = partes[1]

        # Devolver primer nombre + primera letra del segundo en mayúscula + punto (sin espacios)
        return f"{primer_nombre}{segunda_parte[0].upper()}."

    return nombre_completo


# =====================================================
# FUNCIONES DE PRESENTACIÓN (SEPARAR DATOS DE VISTA)
# =====================================================


def get_devoluciones_icon(count):
    """
    Convierte número de devoluciones a indicador visual con emoji
    """
    return f"{count} {'🔴' if count >= 5 else '🟡' if count >= 3 else '🟢'}"


def get_codigo_comentarios_icon(count):
    """
    Convierte número de comentarios de código a indicador visual con emoji
    """
    return f"{count} {'🔴' if count >= 10 else '🟡' if count >= 4 else '🟢'}"


def get_codigo_comentarios_category(count):
    """
    Categoriza comentarios de código para análisis (sin emojis)
    """
    if count >= 10:
        return "Alto"
    elif count >= 4:
        return "Medio"
    else:
        return "Bajo"


def get_tiempo_resolucion_icon(dias):
    """
    Convierte días de resolución a indicador visual con emoji (solo muestra ícono si es lento)
    """
    return f"🐌 {dias}" if dias >= 7 else str(dias)


def get_tiempo_resolucion_category(dias):
    """
    Categoriza tiempo de resolución para análisis: solo "Normal" y "Lento"
    """
    if dias >= 7:
        return "Lento"
    else:
        return "Normal"


print("✅ Funciones de utilidad y presentación definidas")

In [ ]:
# Función para obtener participantes de un pull request
def get_pr_participants(workspace, repo_slug, pr_id):
    """
    Obtiene los participantes de un pull request específico con sus estados de revisión
    """
    participants_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}/participants"
    participants = get_all_items(participants_url)
    return participants

# Función para obtener información detallada de un pull request
def get_pr_details(workspace, repo_slug, pr_id):
    """
    Obtiene información detallada de un pull request específico
    """
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}"
    response = requests.get(pr_url, auth=(username, app_password))
    if response.status_code == 200:
        return response.json()
    return None

# Función para obtener información detallada de un pull request incluyendo participants
def get_pr_details_with_participants(workspace, repo_slug, pr_id):
    """
    Obtiene información detallada de un pull request específico incluyendo participants
    """
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests/{pr_id}"
    response = requests.get(pr_url, auth=(username, app_password))
    if response.status_code == 200:
        return response.json()
    return None

# Función para analizar los estados de revisión de los reviewers
def analyze_reviewer_statuses(participants, reviewers_basic):
    """
    Analiza los estados de revisión de los participantes y reviewers
    Retorna conteos de aprobaciones y solicitudes de cambio
    """
    approvals = 0
    changes_requested = 0
    reviewers_info = []
    
    # Crear un diccionario para facilitar la búsqueda
    participants_dict = {}
    for participant in participants:
        if participant.get('user'):
            username = participant['user'].get('nickname') or participant['user'].get('display_name')
            if username:
                participants_dict[username] = participant
    
    # Analizar cada reviewer
    for reviewer in reviewers_basic:
        reviewer_info = {"name": reviewer, "status": "pending"}
        
        # Buscar el reviewer en los participantes
        if reviewer in participants_dict:
            participant = participants_dict[reviewer]
            if participant.get('approved'):
                approvals += 1
                reviewer_info["status"] = "approved"
            elif participant.get('state') == 'changes_requested':
                changes_requested += 1
                reviewer_info["status"] = "changes_requested"
        
        reviewers_info.append(reviewer_info)
    
    return approvals, changes_requested, reviewers_info

# Función para analizar los estados de revisión desde los datos del PR
def analyze_pr_review_statuses(pr_data):
    """
    Analiza los estados de revisión desde los datos completos del PR
    Extrae información de reviewers y participants si está disponible
    """
    approvals = 0
    changes_requested = 0
    reviewers_info = []
    
    # Primero obtener reviewers básicos
    reviewers_basic = []
    if "reviewers" in pr_data and pr_data["reviewers"]:
        for reviewer in pr_data["reviewers"]:
            if "nickname" in reviewer:
                reviewers_basic.append(reviewer["nickname"])
            elif "display_name" in reviewer:
                reviewers_basic.append(reviewer["display_name"])
    
    # Buscar información de participants si está disponible en el PR
    participants = pr_data.get("participants", [])
    
    # Crear diccionario de participantes para facilitar búsqueda
    participants_dict = {}
    for participant in participants:
        if participant.get('user'):
            user_name = participant['user'].get('nickname') or participant['user'].get('display_name')
            if user_name:
                participants_dict[user_name] = participant
    
    # Analizar cada reviewer
    for reviewer in reviewers_basic:
        reviewer_info = {"name": reviewer, "status": "pending"}
        
        # Buscar el reviewer en los participantes
        if reviewer in participants_dict:
            participant = participants_dict[reviewer]
            # Verificar estados de aprobación
            if participant.get('approved'):
                approvals += 1
                reviewer_info["status"] = "approved"
            elif participant.get('state') == 'changes_requested':
                changes_requested += 1
                reviewer_info["status"] = "changes_requested"
        
        reviewers_info.append(reviewer_info)
    
    return approvals, changes_requested, reviewers_info, reviewers_basic

# Función para analizar todos los participants
def analyze_participants_reviewers(pr_data):
    """
    Analiza todos los participants del PR y extrae su estado de aprobación
    - Aprobado: cuando 'approved' es True
    - No aprobado/Pendiente: cuando 'state' es 'changes_requested'
    """
    participants_reviewers = []
    
    # Obtener participants del PR
    participants = pr_data.get("participants", [])
    
    # Procesar todos los participants (sin filtrar por role)
    for participant in participants:
        user_info = participant.get('user', {})
        user_name = user_info.get('display_name') or user_info.get('nickname', 'Usuario Desconocido')
        
        # Verificar el estado según las nuevas reglas
        is_approved = participant.get('approved', False)
        state = participant.get('state', '')
        
        participant_info = {
            "name": user_name,
            "approved": is_approved,
            "state": state
        }
        
        participants_reviewers.append(participant_info)
    
    return participants_reviewers

print("✅ Funciones actualizadas para usar todos los participants con nuevas reglas de estado")

In [ ]:
# Obtener lista de repositorios
url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

print("🔄 Obteniendo lista de repositorios...")
repositorios_data = get_all_items(url_repos)

if repositorios_data:
    repos_list = []
    for repo in repositorios_data:
        repos_list.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
                "Descripción": repo.get("description", ""),
                "Privado": repo.get("is_private", False),
            }
        )

    repos_df = pd.DataFrame(repos_list)
    repos_df = repos_df.sort_values(by="Nombre")

    # Filtrar repositorios (excluir algunos como en el archivo original)
    repositorios_excluir = ["pgp", "Pruebas_erp", "Inventario", "b2c", "efi"]
    repositorios_activos = [
        repo["Nombre"]
        for repo in repos_list
        if repo["Nombre"] not in repositorios_excluir
    ]

    print(f"✅ Total de repositorios encontrados: {len(repositorios_data)}")
    print(f"✅ Repositorios activos para análisis: {len(repositorios_activos)}")

    # Mostrar algunos repositorios
    display(repos_df.head(10))
else:
    print("❌ No se pudieron obtener los repositorios")

## 🔍 Obtener Pull Requests Merged

In [ ]:
# Función para obtener PRs de un repositorio individual
def obtener_prs_repositorio(repo_slug, workspace, repo_index, total_repos):
    """
    Obtiene los pull requests merged de un repositorio específico.
    Esta función está diseñada para ser ejecutada en paralelo.
    """
    try:
        print(f"   🔄 Procesando repositorio {repo_index}/{total_repos}: {repo_slug}")
        
        # URL para obtener PRs merged
        pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests"
        
        # Obtener solo PRs merged (sin declined)
        params = {"state": "MERGED"}
        prs = get_all_items(pr_url, params=params)
        
        # Limitar a los últimos 50 PRs por repositorio para optimizar
        if len(prs) > 50:
            prs = prs[:50]
        
        # Agregar información del repositorio a cada PR
        for pr in prs:
            pr["repository"] = repo_slug
        
        print(f"   ✅ Repositorio {repo_slug}: {len(prs)} PRs encontrados")
        return prs
        
    except Exception as e:
        print(f"   ❌ Error procesando repositorio {repo_slug}: {str(e)}")
        return []

# Obtener todos los pull requests merged EN PARALELO
print("🔄 Obteniendo pull requests merged de todos los repositorios...")
print("🚀 Usando procesamiento paralelo para acelerar las consultas...")

import time
from concurrent.futures import ThreadPoolExecutor

start_time = time.time()
pull_requests_merged = []
total_repos = len(repositorios_activos)

# Usar ThreadPoolExecutor para consultas paralelas
with ThreadPoolExecutor(max_workers=8) as executor:
    # Enviar todas las tareas al pool de threads
    futures = []
    for i, repo_slug in enumerate(repositorios_activos, 1):
        future = executor.submit(obtener_prs_repositorio, repo_slug, workspace, i, total_repos)
        futures.append(future)
    
    # Recopilar resultados conforme van completándose
    for future in futures:
        repo_prs = future.result()
        if repo_prs:  # Solo agregar si hay PRs encontrados
            pull_requests_merged.extend(repo_prs)

# Calcular tiempo de consulta
end_time = time.time()
tiempo_consulta = end_time - start_time

print(f"\n✅ Consulta paralela completada en {tiempo_consulta:.1f} segundos")
print(f"✅ Total de pull requests merged encontrados: {len(pull_requests_merged)}")
print(f"🚀 Mejora de rendimiento: {(total_repos * 2 / tiempo_consulta):.1f}x más rápido estimado")

if len(pull_requests_merged) == 0:
    print("ℹ️  No hay pull requests merged en el período consultado")
else:
    print("🔄 Procesando información adicional de los pull requests...")

    # Filtrar por fecha si es necesario (últimos 30 días por defecto)
    from datetime import datetime, timedelta

    fecha_limite = datetime.now() - timedelta(days=30)

    # Filtrar PRs por fecha de merge
    prs_filtrados = []
    for pr in pull_requests_merged:
        fecha_updated = pd.to_datetime(pr["updated_on"])
        if fecha_updated.tz_localize(None) >= fecha_limite:
            prs_filtrados.append(pr)

    pull_requests_merged = prs_filtrados
    print(f"🔄 PRs filtrados por últimos 30 días: {len(pull_requests_merged)}")

In [ ]:
# Función para procesar un PR individual de manera thread-safe
def procesar_pr_individual(pr, progress_dict, total_prs):
    try:
        # Extraer información del autor del PR
        autor = "Desconocido"
        if "author" in pr and pr["author"]:
            if "display_name" in pr["author"]:
                autor = pr["author"]["display_name"]
            elif "nickname" in pr["author"]:
                autor = pr["author"]["nickname"]
        autor = acortar_nombre(autor)

        # Obtener comentarios del PR y analizarlos
        try:
            comentarios = get_pr_comments(workspace, pr["repository"], pr["id"])
            comm_pullrequest, estandarizaciones_codigo = analyze_pr_comments(
                comentarios, autor
            )
        except Exception as e:
            print(f"      ⚠️  Error obteniendo comentarios para PR {pr['id']}: {str(e)}")
            comm_pullrequest = 0
            estandarizaciones_codigo = 0

        # Calcular tiempo de resolución
        created_on = pd.to_datetime(pr["created_on"])
        updated_on = pd.to_datetime(pr["updated_on"])
        tiempo_resolucion = (updated_on - created_on).days

        # Información sobre quien merged el PR
        merged_por = "Desconocido"
        if "closed_by" in pr and pr["closed_by"]:
            if "display_name" in pr["closed_by"]:
                merged_por = pr["closed_by"]["display_name"]
            elif "nickname" in pr["closed_by"]:
                merged_por = pr["closed_by"]["nickname"]

        # Crear registro procesado
        pr_procesado = {
            "ID": pr["id"],
            "Título": pr.get("title", "Sin título"),
            "Repositorio": pr["repository"],
            "Autor": autor,
            "Merged_Por": merged_por,
            "Devoluciones": comm_pullrequest,
            "Estandarizaciones_Codigo": estandarizaciones_codigo,
            "Fecha_Creacion": created_on,
            "Fecha_Merge": updated_on,
            "Tiempo_Resolucion": tiempo_resolucion,
            "Descripcion": (
                pr.get("description", "Sin descripción")[:100] + "..."
                if pr.get("description", "")
                else "Sin descripción"
            ),
            "URL": pr.get("links", {}).get("html", {}).get("href", ""),
            "Rama_Origen": pr.get("source", {})
            .get("branch", {})
            .get("name", "Desconocida"),
            "Rama_Destino": pr.get("destination", {})
            .get("branch", {})
            .get("name", "Desconocida"),
        }

        # Actualizar progreso de manera thread-safe
        with progress_dict['lock']:
            progress_dict['procesados'] += 1
            current = progress_dict['procesados']
            print(f"   ✅ PR {current}/{total_prs} procesado: {pr.get('title', 'Sin título')[:50]}...")

        return pr_procesado

    except Exception as e:
        print(f"   ❌ Error procesando PR {pr.get('id', 'unknown')}: {str(e)}")
        return None

# Procesar información detallada de cada pull request merged con threading
if len(pull_requests_merged) > 0:
    print(f"🚀 Iniciando procesamiento con threading de {len(pull_requests_merged)} PRs...")
    
    # Medir tiempo de procesamiento
    start_time = time.time()
    
    # Variables para tracking del progreso thread-safe
    progress_dict = {
        'procesados': 0,
        'lock': threading.Lock()
    }
    
    # Usar ThreadPoolExecutor para procesamiento paralelo
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Enviar todas las tareas al pool de threads
        futures = []
        for pr in pull_requests_merged:
            future = executor.submit(procesar_pr_individual, pr, progress_dict, len(pull_requests_merged))
            futures.append(future)
        
        # Recopilar resultados
        prs_procesados = []
        for future in futures:
            resultado = future.result()
            if resultado is not None:
                prs_procesados.append(resultado)
    
    # Calcular tiempo transcurrido
    end_time = time.time()
    tiempo_total = end_time - start_time
    
    # Crear DataFrame con los datos procesados
    df_prs_merged = pd.DataFrame(prs_procesados)

    # Ordenar por fecha de merge (más recientes primero)
    df_prs_merged = df_prs_merged.sort_values("Fecha_Merge", ascending=False)

    print(f"\n✅ Procesamiento completado en {tiempo_total:.1f} segundos")
    print(f"📊 {len(df_prs_merged)} pull requests procesados exitosamente")
    if len(prs_procesados) < len(pull_requests_merged):
        print(f"⚠️  {len(pull_requests_merged) - len(prs_procesados)} PRs fallaron en el procesamiento")
else:
    df_prs_merged = pd.DataFrame()

## 📊 Reportes y Análisis

In [ ]:
# Resumen ejecutivo
if not df_prs_merged.empty:
    print("=" * 60)
    print("📈 RESUMEN EJECUTIVO - PULL REQUESTS MERGED")
    print("=" * 60)

    total_prs = len(df_prs_merged)

    prs_con_comentarios = len(df_prs_merged[df_prs_merged["Devoluciones"] > 0])
    total_comentarios = df_prs_merged["Devoluciones"].sum()
    total_estandarizaciones_codigo = df_prs_merged["Estandarizaciones_Codigo"].sum()

    promedio_tiempo_resolucion = df_prs_merged["Tiempo_Resolucion"].mean()
    max_tiempo_resolucion = df_prs_merged["Tiempo_Resolucion"].max()
    min_tiempo_resolucion = df_prs_merged["Tiempo_Resolucion"].min()

    print(f"🔢 Total de Pull Requests Merged: {total_prs}")
    print(f"💬 PRs con Comentarios: {prs_con_comentarios}")
    print(f"📊 Total de Comentarios: {total_comentarios}")
    print(
        f"👨‍💻 Estandarizaciones de Código (inline): {total_estandarizaciones_codigo}"
    )
    print(f"⏱️  Tiempo Promedio de Resolución: {promedio_tiempo_resolucion:.1f} días")
    print(f"🚀 Resolución más Rápida: {min_tiempo_resolucion} días")
    print(f"🐌 Resolución más Lenta: {max_tiempo_resolucion} días")

    # Repositorios con más PRs merged
    repos_prs = df_prs_merged["Repositorio"].value_counts()
    print(f"\n🏆 Repositorios con más PRs merged:")
    for i, (repo, count) in enumerate(repos_prs.head(5).items(), 1):
        print(f"   {i}. {repo}: {count} PRs")

    # Autores con más PRs merged
    autores_prs = df_prs_merged["Autor"].value_counts()
    print(f"\n👥 Autores con más PRs merged:")
    for i, (autor, count) in enumerate(autores_prs.head(5).items(), 1):
        print(f"   {i}. {autor}: {count} PRs")

    # Personas que más hacen merge
    merged_por = df_prs_merged["Merged_Por"].value_counts()
    print(f"\n🔀 Quienes más hacen merge de PRs:")
    for i, (persona, count) in enumerate(merged_por.head(5).items(), 1):
        print(f"   {i}. {persona}: {count} PRs")

    print("=" * 60)
else:
    print(
        "ℹ️  No hay pull requests merged en el período consultado para mostrar en el resumen"
    )

In [ ]:
# Mostrar tabla detallada de Pull Requests merged
if not df_prs_merged.empty:
    print("\n📋 DETALLE DE PULL REQUESTS MERGED")
    print("=" * 80)

    # Mostrar rango temporal de los datos analizados
    fecha_primer_pr = df_prs_merged["Fecha_Creacion"].min()
    fecha_ultimo_pr = df_prs_merged["Fecha_Creacion"].max()

    print(f"📅 Fecha primer pull request: {fecha_primer_pr.strftime('%d/%m/%Y %H:%M')}")
    print(f"📅 Fecha último pull request: {fecha_ultimo_pr.strftime('%d/%m/%Y %H:%M')}")

    # Calcular período total
    periodo_total = (fecha_ultimo_pr - fecha_primer_pr).days
    print(
        f"⏱️  Período total analizado: {periodo_total} días ({periodo_total / 30:.1f} meses)"
    )
    print("=" * 80)

    # Crear una versión de la tabla optimizada para visualización (sin Merged_Por)
    df_display = df_prs_merged[
        [
            "ID",
            "Repositorio",
            "Título",
            "Autor",
            "Devoluciones",
            "Estandarizaciones_Codigo",
            "Tiempo_Resolucion",
            "Rama_Origen",
            "Fecha_Creacion",
            "Fecha_Merge",
        ]
    ].copy()

    # Ordenar por fecha de merge (más recientes primero)
    df_display = df_display.sort_values("Fecha_Merge", ascending=False)

    # Convertir fechas a formato "para humanos"
    df_display["Creado"] = df_display["Fecha_Creacion"].apply(fecha_para_humanos)
    df_display["Merged"] = df_display["Fecha_Merge"].apply(fecha_para_humanos)

    # Eliminar las columnas de fecha originales
    df_display = df_display.drop(columns=["Fecha_Creacion", "Fecha_Merge"])

    # ===================================================================
    # MEJORES PRÁCTICAS: Crear columnas categóricas para análisis futuro
    # ===================================================================

    # Categorizar comentarios de código (para análisis/filtros)
    df_display["Estandarizaciones_Codigo_Categoria"] = df_display[
        "Estandarizaciones_Codigo"
    ].apply(get_codigo_comentarios_category)

    # Categorizar tiempo de resolución (para análisis/filtros) - Solo "Normal" y "Lento"
    df_display["Tiempo_Resolucion_Categoria"] = df_display["Tiempo_Resolucion"].apply(
        get_tiempo_resolucion_category
    )

    # Renombrar columnas para mayor claridad
    df_display = df_display.rename(columns={"Rama_Origen": "Branch"})

    # ===================================================================
    # APLICAR PRESENTACIÓN VISUAL SOLO PARA MOSTRAR (NO PARA ALMACENAR)
    # ===================================================================

    # Crear copia temporal solo para visualización
    df_show = df_display.copy()

    # Aplicar iconos solo en la copia de visualización
    df_show["Devoluciones"] = df_show["Devoluciones"].apply(get_devoluciones_icon)

    # Aplicar iconos solo en la copia de visualización
    df_show["Estandarizaciones_Codigo"] = df_show["Estandarizaciones_Codigo"].apply(
        get_codigo_comentarios_icon
    )
    df_show["Tiempo_Resolucion"] = df_show["Tiempo_Resolucion"].apply(
        get_tiempo_resolucion_icon
    )

    # Renombrar columna de tiempo para mostrar
    df_show = df_show.rename(columns={"Tiempo_Resolucion": "Días_Resolución"})

    # Seleccionar columnas para mostrar (sin las categóricas internas)
    df_show = df_show[
        [
            "ID",
            "Repositorio",
            "Título",
            "Autor",
            "Branch",
            "Devoluciones",
            "Estandarizaciones_Codigo",
            "Días_Resolución",
            "Creado",
            "Merged",
        ]
    ]

    # Configurar pandas para mostrar todas las columnas
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_colwidth", 50)

    # Mostrar la tabla (solo la versión con iconos)
    display(df_show)

    print(f"\n📌 Mostrando {len(df_display)} pull requests merged")
    print("💡 Tip: Los PRs están ordenados por fecha de merge (más recientes primero)")
    print("🐌 Resolución lenta: ≥7 días (sin ícono = resolución normal)")
    print("🟢 0-2 devoluciones | 🟡 3-4 comentarios | 🔴 5+ comentarios")
    print("🟢 0-3 comentarios de código | 🟡 4-9 comentarios | 🔴 10+ comentarios")

    print(f"\n🔍 Para análisis futuro usar:")
    print(
        f"   • df_display['Estandarizaciones_Codigo_Categoria'] -> 'Bajo', 'Medio', 'Alto'"
    )
    print(f"   • df_display['Tiempo_Resolucion_Categoria'] -> 'Normal', 'Lento'")
    print(f"   • df_display['Estandarizaciones_Codigo'] -> valores numéricos puros")
    print(f"   • df_display['Tiempo_Resolucion'] -> días numéricos puros")
else:
    print("ℹ️  No hay pull requests merged en el período consultado")

In [ ]:
# Análisis específico de los datos actuales para generar recomendaciones de reportes
if not df_prs_merged.empty:
    print("🔍 ANÁLISIS DE DATOS DISPONIBLES Y RECOMENDACIONES DE REPORTES")
    print("=" * 70)

    # Análisis básico de los datos
    total_prs = len(df_prs_merged)
    autores_unicos = df_prs_merged["Autor"].nunique()
    repos_unicos = df_prs_merged["Repositorio"].nunique()

    print(f"📊 Resumen de datos disponibles:")
    print(f"   • Total PRs: {total_prs}")
    print(f"   • Autores únicos: {autores_unicos}")
    print(f"   • Repositorios únicos: {repos_unicos}")
    print(
        f"   • Rango de fechas: {df_prs_merged['Fecha_Creacion'].min().strftime('%Y-%m-%d')} a {df_prs_merged['Fecha_Merge'].max().strftime('%Y-%m-%d')}"
    )

    # Análisis de distribución de métricas clave
    print(f"\n📈 Distribución de métricas clave:")

    # Estandarizaciones de código
    est_stats = df_prs_merged["Estandarizaciones_Codigo"].describe()
    print(f"   🔧 Estandarizaciones de Código:")
    print(f"      - Promedio: {est_stats['mean']:.1f}")
    print(f"      - Mediana: {est_stats['50%']:.1f}")
    print(f"      - Máximo: {int(est_stats['max'])}")

    # Devoluciones
    dev_stats = df_prs_merged["Devoluciones"].describe()
    print(f"   🔄 Devoluciones:")
    print(f"      - Promedio: {dev_stats['mean']:.1f}")
    print(f"      - Mediana: {dev_stats['50%']:.1f}")
    print(f"      - Máximo: {int(dev_stats['max'])}")

    # Tiempo de resolución
    tiempo_stats = df_prs_merged["Tiempo_Resolucion"].describe()
    print(f"   ⏱️  Tiempo de Resolución (días):")
    print(f"      - Promedio: {tiempo_stats['mean']:.1f}")
    print(f"      - Mediana: {tiempo_stats['50%']:.1f}")
    print(f"      - Máximo: {int(tiempo_stats['max'])}")

    print(f"\n🎯 REPORTES RECOMENDADOS POR PRIORIDAD:")
    print("=" * 50)

    # Generar recomendaciones basadas en los datos
    reportes_recomendados = []

    # 1. Si hay variabilidad en autores, recomendar análisis de productividad
    if autores_unicos >= 3:
        reportes_recomendados.append(
            {
                "prioridad": "🔴 ALTA",
                "nombre": "Dashboard de Productividad por Desarrollador",
                "justificacion": f"Con {autores_unicos} desarrolladores activos, este reporte identificará patrones de rendimiento y oportunidades de mejora.",
                "impacto": "Gestión de equipo y desarrollo profesional",
            }
        )

    # 2. Si hay alta variabilidad en estandarizaciones, recomendar análisis de calidad
    if est_stats["std"] > 2:
        reportes_recomendados.append(
            {
                "prioridad": "🔴 ALTA",
                "nombre": "Análisis de Calidad de Código",
                "justificacion": f"Alta variabilidad en estandarizaciones (std: {est_stats['std']:.1f}) sugiere inconsistencias en calidad.",
                "impacto": "Mejora de procesos de code review",
            }
        )

    # 3. Si hay múltiples repositorios, recomendar análisis por repo
    if repos_unicos >= 3:
        reportes_recomendados.append(
            {
                "prioridad": "🟡 MEDIA",
                "nombre": "Ranking de Repositorios por Métricas",
                "justificacion": f"Con {repos_unicos} repositorios, podemos identificar cuáles necesitan más atención.",
                "impacto": "Priorización de esfuerzos de mejora técnica",
            }
        )

    # 4. Si hay PRs lentos, recomendar análisis de eficiencia
    prs_lentos = len(df_prs_merged[df_prs_merged["Tiempo_Resolucion"] > 7])
    if prs_lentos > 0:
        porcentaje_lentos = (prs_lentos / total_prs) * 100
        reportes_recomendados.append(
            {
                "prioridad": "🟡 MEDIA" if porcentaje_lentos < 20 else "🔴 ALTA",
                "nombre": "Análisis de Eficiencia del Proceso",
                "justificacion": f"{prs_lentos} PRs ({porcentaje_lentos:.1f}%) tardaron más de 7 días en resolverse.",
                "impacto": "Optimización del flujo de trabajo",
            }
        )

    # 5. Análisis temporal siempre útil
    reportes_recomendados.append(
        {
            "prioridad": "🟢 BAJA",
            "nombre": "Tendencias Temporales",
            "justificacion": "Útil para identificar patrones estacionales y planificación futura.",
            "impacto": "Planificación estratégica",
        }
    )

    # Mostrar recomendaciones
    for i, reporte in enumerate(reportes_recomendados, 1):
        print(f"\n{i}. {reporte['prioridad']} - {reporte['nombre']}")
        print(f"   💡 Justificación: {reporte['justificacion']}")
        print(f"   🎯 Impacto: {reporte['impacto']}")

    print(f"\n🚀 PRÓXIMOS PASOS SUGERIDOS:")
    print("=" * 40)
    print("1. Comenzar con reportes de prioridad ALTA")
    print("2. Implementar dashboards automáticos para seguimiento continuo")
    print("3. Definir umbrales de alerta para intervención proactiva")
    print("4. Establecer revisiones periódicas de métricas con el equipo")

    # Mostrar información sobre campos disponibles para cada reporte
    print(f"\n📋 CAMPOS DISPONIBLES PARA ANÁLISIS:")
    print("=" * 40)
    campos_numericos = ["Devoluciones", "Estandarizaciones_Codigo", "Tiempo_Resolucion"]
    campos_categoricos = [
        "Autor",
        "Repositorio",
        "Merged_Por",
        "Rama_Origen",
        "Rama_Destino",
    ]
    campos_temporales = ["Fecha_Creacion", "Fecha_Merge"]
    campos_derivados = [
        "Estandarizaciones_Codigo_Categoria",
        "Tiempo_Resolucion_Categoria",
    ]

    print(f"🔢 Campos numéricos: {', '.join(campos_numericos)}")
    print(f"🏷️  Campos categóricos: {', '.join(campos_categoricos)}")
    print(f"📅 Campos temporales: {', '.join(campos_temporales)}")
    print(f"📊 Campos derivados: {', '.join(campos_derivados)}")

else:
    print("⚠️  No hay datos disponibles para generar recomendaciones de reportes")

In [ ]:
# Verificar las columnas disponibles en df_prs_merged
print("📋 COLUMNAS DISPONIBLES EN df_prs_merged:")
print(list(df_prs_merged.columns))
print(f"\nShape del DataFrame: {df_prs_merged.shape}")

# Verificar si existe la columna 'Autor' en lugar de 'author_name'
if 'Autor' in df_prs_merged.columns:
    print("✅ La columna 'Autor' está disponible")
    print(f"Autores únicos: {df_prs_merged['Autor'].nunique()}")
else:
    print("❌ La columna 'Autor' no está disponible")

# Verificar otras columnas necesarias
columnas_necesarias = ['Tiempo_Resolucion', 'Devoluciones', 'Estandarizaciones_Codigo', 'Fecha_Creacion']
for col in columnas_necesarias:
    if col in df_prs_merged.columns:
        print(f"✅ {col} disponible")
    else:
        print(f"❌ {col} NO disponible")

In [ ]:
# Verificar si existe declined_prs, si no, crear un DataFrame vacío
if 'declined_prs' not in globals():
    declined_prs = pd.DataFrame()  # DataFrame vacío para evitar errores
    print("⚠️ declined_prs no encontrado, creando DataFrame vacío")
else:
    print(f"✅ declined_prs encontrado con {len(declined_prs)} registros")

# 📈 REPORTES Y VISUALIZACIONES IMPLEMENTADAS

En esta sección implementamos los reportes prioritarios identificados en el análisis anterior, proporcionando visualizaciones y métricas que aportan valor directo al equipo de desarrollo para la toma de decisiones y mejora continua de procesos.

## 🎯 REPORTE 1: Dashboard de Productividad del Equipo

**Objetivo:** Proporcionar una vista consolidada de la productividad del equipo con métricas clave de rendimiento.

**Métricas incluidas:**
- Volumen total de PRs procesados por desarrollador
- Tiempo promedio de resolución por desarrollador
- Tasa de éxito (PRs merged vs declined)
- Distribución de carga de trabajo
- Ranking de productividad balanceada (velocidad + volumen)

In [ ]:
# ====================================================================================
# REPORTE 1: DASHBOARD DE PRODUCTIVIDAD DEL EQUIPO
# ====================================================================================


# Configurar el estilo de las visualizaciones
plt.style.use("default")
sns.set_palette("husl")

# Preparar datos para el dashboard de productividad
print("🚀 DASHBOARD DE PRODUCTIVIDAD DEL EQUIPO")
print("=" * 60)

# 1. Métricas generales de productividad
metrics_productividad = {
    "Total PRs Procesados": len(df_prs_merged),
    "Desarrolladores Activos": len(df_prs_merged["Autor"].unique()),
    "Tiempo Promedio Resolución": f"{df_prs_merged['Tiempo_Resolucion'].mean():.1f} días",
    "Tasa de Éxito Global": f"{(len(df_prs_merged) / (len(df_prs_merged) + len(declined_prs)) * 100):.1f}%",
    "PRs/Día (promedio)": f"{len(df_prs_merged) / ((df_prs_merged['Fecha_Creacion'].max() - df_prs_merged['Fecha_Creacion'].min()).days + 1):.1f}",
}

# Mostrar métricas principales
print("\n📊 MÉTRICAS PRINCIPALES:")
for metric, value in metrics_productividad.items():
    print(f"   {metric}: {value}")

# 2. Análisis de productividad por desarrollador
print("\n\n👥 ANÁLISIS POR DESARROLLADOR:")

# Calcular métricas detalladas por autor
productivity_stats = (
    df_prs_merged.groupby("Autor")
    .agg(
        {
            "ID": "count",  # Volumen de PRs
            "Tiempo_Resolucion": ["mean", "std"],  # Tiempo promedio y variabilidad
            "Devoluciones": "mean",  # Complejidad promedio
            "Estandarizaciones_Codigo": "mean",  # Calidad del código
        }
    )
    .round(2)
)

# Aplanar nombres de columnas
productivity_stats.columns = [
    "PRs_Total",
    "Tiempo_Promedio",
    "Tiempo_Std",
    "Devoluciones_Promedio",
    "Estandarizaciones_Promedio",
]

# Calcular score de productividad (balanceado entre volumen y velocidad)
productivity_stats["Score_Volumen"] = (
    productivity_stats["PRs_Total"] / productivity_stats["PRs_Total"].max()
) * 100
productivity_stats["Score_Velocidad"] = (
    1
    - (
        productivity_stats["Tiempo_Promedio"]
        / productivity_stats["Tiempo_Promedio"].max()
    )
) * 100
productivity_stats["Score_Productividad"] = (
    productivity_stats["Score_Volumen"] + productivity_stats["Score_Velocidad"]
) / 2

# Ordenar por score de productividad
productivity_stats = productivity_stats.sort_values(
    "Score_Productividad", ascending=False
)

# Mostrar top 10 desarrolladores más productivos
print(f"\n🏆 TOP 10 DESARROLLADORES MÁS PRODUCTIVOS:")
display(
    productivity_stats.head(10)[["PRs_Total", "Tiempo_Promedio", "Score_Productividad"]]
)

# 3. Crear visualización del dashboard
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(4, 4, hspace=0.7, wspace=0.3)

# Gráfico 1: Volumen de PRs por desarrollador (Top 15)
ax1 = fig.add_subplot(gs[0, :2])
top_15_volume = productivity_stats.head(15)
bars1 = ax1.bar(
    range(len(top_15_volume)), top_15_volume["PRs_Total"], color="steelblue", alpha=0.8
)
ax1.set_title(
    "📊 Volumen de PRs por Desarrollador (Top 15)", fontsize=14, fontweight="bold"
)
ax1.set_ylabel("Número de PRs")
ax1.set_xticks(range(len(top_15_volume)))
ax1.set_xticklabels(
    [name[:15] + "..." if len(name) > 15 else name for name in top_15_volume.index],
    rotation=45,
    ha="right",
)

# Agregar etiquetas en las barras
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.5,
        f"{int(height)}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

# Gráfico 2: Tiempo promedio de resolución (Top 15)
ax2 = fig.add_subplot(gs[0, 2:])
bars2 = ax2.bar(
    range(len(top_15_volume)),
    top_15_volume["Tiempo_Promedio"],
    color="lightcoral",
    alpha=0.8,
)
ax2.set_title(
    "⏱️ Tiempo Promedio de Resolución (Top 15)", fontsize=14, fontweight="bold"
)
ax2.set_ylabel("Días")
ax2.set_xticks(range(len(top_15_volume)))
ax2.set_xticklabels(
    [name[:15] + "..." if len(name) > 15 else name for name in top_15_volume.index],
    rotation=45,
    ha="right",
)

# Agregar etiquetas en las barras
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.1,
        f"{height:.1f}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

# Gráfico 3: Score de Productividad (Top 12)
ax3 = fig.add_subplot(gs[1, :2])
top_12_prod = productivity_stats.head(12)
colors_grad = plt.cm.RdYlGn([x / 100 for x in top_12_prod["Score_Productividad"]])
bars3 = ax3.bar(
    range(len(top_12_prod)),
    top_12_prod["Score_Productividad"],
    color=colors_grad,
    alpha=0.8,
)
ax3.set_title(
    "🏆 Score de Productividad Balanceado (Top 12)", fontsize=14, fontweight="bold"
)
ax3.set_ylabel("Score (0-100)")
ax3.set_xticks(range(len(top_12_prod)))
ax3.set_xticklabels(
    [name[:12] + "..." if len(name) > 12 else name for name in top_12_prod.index],
    rotation=45,
    ha="right",
)

# Agregar etiquetas en las barras
for i, bar in enumerate(bars3):
    height = bar.get_height()
    ax3.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 1,
        f"{height:.1f}",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

# Gráfico 4: Distribución de carga de trabajo
ax4 = fig.add_subplot(gs[1, 2:])
# Agrupar desarrolladores por rangos de volumen
bins = [0, 5, 15, 30, 50, 999]
labels = ["1-5 PRs", "6-15 PRs", "16-30 PRs", "31-50 PRs", "50+ PRs"]
productivity_stats["Rango_Volumen"] = pd.cut(
    productivity_stats["PRs_Total"], bins=bins, labels=labels, include_lowest=True
)
dist_carga = productivity_stats["Rango_Volumen"].value_counts()

wedges, texts, autotexts = ax4.pie(
    dist_carga.values,
    labels=dist_carga.index,
    autopct="%1.1f%%",
    colors=sns.color_palette("Set3", len(dist_carga)),
)
ax4.set_title("📈 Distribución de Carga de Trabajo", fontsize=14, fontweight="bold")

# Mejorar legibilidad del pie chart
for autotext in autotexts:
    autotext.set_color("white")
    autotext.set_fontweight("bold")

# Gráfico 5: Correlación Volumen vs Velocidad
ax5 = fig.add_subplot(gs[2, :2])
scatter = ax5.scatter(
    productivity_stats["PRs_Total"],
    productivity_stats["Tiempo_Promedio"],
    c=productivity_stats["Score_Productividad"],
    cmap="RdYlGn",
    s=100,
    alpha=0.7,
    edgecolors="black",
    linewidth=0.5,
)
ax5.set_xlabel("Volumen de PRs")
ax5.set_ylabel("Tiempo Promedio (días)")
ax5.set_title("🎯 Relación Volumen vs Velocidad", fontsize=14, fontweight="bold")

# Agregar colorbar
cbar = plt.colorbar(scatter, ax=ax5)
cbar.set_label("Score de Productividad", rotation=270, labelpad=20)

# Agregar línea de tendencia
z = np.polyfit(
    productivity_stats["PRs_Total"], productivity_stats["Tiempo_Promedio"], 1
)
p = np.poly1d(z)
ax5.plot(
    productivity_stats["PRs_Total"],
    p(productivity_stats["PRs_Total"]),
    "r--",
    alpha=0.8,
    linewidth=2,
)

# Gráfico 6: Evolución temporal de productividad (últimos 6 meses)
ax6 = fig.add_subplot(gs[2, 2:])
if len(df_prs_merged) > 0:
    # Agrupar por mes
    df_prs_merged["mes"] = df_prs_merged["Fecha_Creacion"].dt.to_period("M")
    productivity_monthly = (
        df_prs_merged.groupby("mes")
        .agg({"ID": "count", "Tiempo_Resolucion": "mean"})
        .reset_index()
    )

    # Mostrar últimos 6 meses
    productivity_monthly = productivity_monthly.tail(6)

    ax6_twin = ax6.twinx()

    # Línea de volumen
    line1 = ax6.plot(
        productivity_monthly["mes"].astype(str),
        productivity_monthly["ID"],
        "b-o",
        linewidth=3,
        markersize=8,
        label="Volumen PRs",
    )
    ax6.set_ylabel("Número de PRs", color="blue")
    ax6.tick_params(axis="y", labelcolor="blue")

    # Línea de tiempo promedio
    line2 = ax6_twin.plot(
        productivity_monthly["mes"].astype(str),
        productivity_monthly["Tiempo_Resolucion"],
        "r-s",
        linewidth=3,
        markersize=8,
        label="Tiempo Promedio",
    )
    ax6_twin.set_ylabel("Tiempo Promedio (días)", color="red")
    ax6_twin.tick_params(axis="y", labelcolor="red")

    ax6.set_title(
        "📅 Evolución Temporal de Productividad", fontsize=14, fontweight="bold"
    )
    ax6.set_xlabel("Mes")
    plt.setp(ax6.xaxis.get_majorticklabels(), rotation=45)

    # Combinar leyendas
    lines1, labels1 = ax6.get_legend_handles_labels()
    lines2, labels2 = ax6_twin.get_legend_handles_labels()
    ax6.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

# Tabla resumen de insights
ax7 = fig.add_subplot(gs[3, :])
ax7.axis("tight")
ax7.axis("off")

# Generar insights automáticos
insights_data = [
    [
        "🏆 Desarrollador Más Productivo",
        productivity_stats.index[0],
        f"{productivity_stats.iloc[0]['Score_Productividad']:.1f} puntos",
    ],
    [
        "⚡ Desarrollador Más Rápido",
        productivity_stats.nsmallest(1, "Tiempo_Promedio").index[0],
        f"{productivity_stats.nsmallest(1, 'Tiempo_Promedio').iloc[0]['Tiempo_Promedio']:.1f} días",
    ],
    [
        "🎯 Desarrollador Más Volumen",
        productivity_stats.nlargest(1, "PRs_Total").index[0],
        f"{productivity_stats.nlargest(1, 'PRs_Total').iloc[0]['PRs_Total']} PRs",
    ],
    [
        "📊 Promedio Equipo",
        "Tiempo de Resolución",
        f"{productivity_stats['Tiempo_Promedio'].mean():.1f} días",
    ],
    [
        "🔍 Recomendación",
        "Desarrolladores a Analizar",
        f"{len(productivity_stats[productivity_stats['Score_Productividad'] < 30])} con score < 30",
    ],
]

table = ax7.table(
    cellText=insights_data,
    colLabels=["Métrica", "Valor", "Detalle"],
    cellLoc="center",
    loc="center",
    colWidths=[0.3, 0.3, 0.4],
)

table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)

# Estilizar tabla
for i in range(len(insights_data) + 1):
    for j in range(3):
        cell = table[(i, j)]
        if i == 0:  # Header
            cell.set_facecolor("#4CAF50")
            cell.set_text_props(weight="bold", color="white")
        else:
            cell.set_facecolor("#f8f9fa" if i % 2 == 0 else "white")

ax7.set_title(
    "💡 INSIGHTS Y RECOMENDACIONES CLAVE", fontsize=16, fontweight="bold", pad=20
)

plt.suptitle(
    "🎯 DASHBOARD DE PRODUCTIVIDAD DEL EQUIPO", fontsize=20, fontweight="bold", y=0.98
)
plt.tight_layout()
plt.show()

print("\n✅ Dashboard de Productividad generado exitosamente!")
print(
    f"📈 Datos analizados: {len(df_prs_merged)} PRs de {len(productivity_stats)} desarrolladores"
)

## 🔍 REPORTE 2: Análisis de Eficiencia y Calidad

**Objetivo:** Evaluar la eficiencia en el proceso de revisión y la calidad del código entregado.

**Métricas incluidas:**
- Tiempo de resolución vs complejidad (comentarios/estandarizaciones)
- Identificación de desarrolladores que necesitan más soporte
- Análisis de patrones de trabajo y outliers
- Correlaciones entre métricas de calidad y eficiencia
- Recomendaciones específicas de mejora

In [ ]:
# ====================================================================================
# REPORTE 2: ANÁLISIS DE EFICIENCIA Y CALIDAD
# ====================================================================================

print("🔍 ANÁLISIS DE EFICIENCIA Y CALIDAD")
print("=" * 60)

# 1. Análisis de correlaciones entre métricas
print("\n📊 ANÁLISIS DE CORRELACIONES ENTRE MÉTRICAS:")

# Preparar datos para correlaciones
correlation_data = df_prs_merged[
    ["Tiempo_Resolucion", "Devoluciones", "Estandarizaciones_Codigo"]
].copy()
correlation_data.columns = [
    "Tiempo_Resolución",
    "Devoluciones",
    "Estandarizaciones_Código",
]

# Calcular matriz de correlación
corr_matrix = correlation_data.corr()
print("\nMatriz de Correlación:")
display(corr_matrix.round(3))

# 2. Análisis de eficiencia por desarrollador
print("\n\n⚡ ANÁLISIS DE EFICIENCIA POR DESARROLLADOR:")

# Calcular métricas de eficiencia
efficiency_stats = (
    df_prs_merged.groupby("Autor")
    .agg(
        {
            "Tiempo_Resolucion": ["mean", "std", "min", "max"],
            "Devoluciones": ["mean", "std"],
            "Estandarizaciones_Codigo": ["mean", "sum"],
            "ID": "count",
        }
    )
    .round(2)
)

# Aplanar nombres de columnas
efficiency_stats.columns = [
    "Tiempo_Medio",
    "Tiempo_Std",
    "Tiempo_Min",
    "Tiempo_Max",
    "Devoluciones_Medio",
    "Devoluciones_Std",
    "Estandarizaciones_Medio",
    "Estandarizaciones_Total",
    "Total_PRs",
]

# Calcular métricas de eficiencia
efficiency_stats["Coef_Variacion_Tiempo"] = (
    efficiency_stats["Tiempo_Std"] / efficiency_stats["Tiempo_Medio"]
) * 100
efficiency_stats["Ratio_Estandarizaciones"] = (
    efficiency_stats["Estandarizaciones_Total"] / efficiency_stats["Total_PRs"]
)
efficiency_stats["Score_Consistencia"] = 100 - efficiency_stats[
    "Coef_Variacion_Tiempo"
].fillna(0)
efficiency_stats["Score_Calidad"] = (
    efficiency_stats["Ratio_Estandarizaciones"]
    / efficiency_stats["Ratio_Estandarizaciones"].max()
) * 100

# Filtrar desarrolladores con al menos 3 PRs para análisis más confiable
efficiency_relevant = efficiency_stats[efficiency_stats["Total_PRs"] >= 3].copy()

# Identificar categorías de desarrolladores
print("\n🎯 CATEGORIZACIÓN DE DESARROLLADORES:")

# Desarrolladores eficientes (rápidos y consistentes)
eficientes = efficiency_relevant[
    (
        efficiency_relevant["Tiempo_Medio"]
        <= efficiency_relevant["Tiempo_Medio"].quantile(0.25)
    )
    & (
        efficiency_relevant["Score_Consistencia"]
        >= efficiency_relevant["Score_Consistencia"].quantile(0.75)
    )
]

# Desarrolladores que necesitan soporte (lentos o inconsistentes)
necesitan_soporte = efficiency_relevant[
    (
        efficiency_relevant["Tiempo_Medio"]
        >= efficiency_relevant["Tiempo_Medio"].quantile(0.75)
    )
    | (
        efficiency_relevant["Score_Consistencia"]
        <= efficiency_relevant["Score_Consistencia"].quantile(0.25)
    )
]

print(f"✅ Desarrolladores Eficientes: {len(eficientes)}")
print(f"⚠️  Desarrolladores que Necesitan Soporte: {len(necesitan_soporte)}")
print(
    f"📊 Desarrolladores Promedio: {len(efficiency_relevant) - len(eficientes) - len(necesitan_soporte)}"
)

# 3. Crear visualización del análisis de eficiencia
fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(4, 3, hspace=0.8, wspace=0.3)

# Gráfico 1: Matriz de correlación
ax1 = fig.add_subplot(gs[0, 0])
im1 = ax1.imshow(corr_matrix, cmap="RdBu_r", aspect="auto", vmin=-1, vmax=1)
ax1.set_xticks(range(len(corr_matrix.columns)))
ax1.set_yticks(range(len(corr_matrix.columns)))
ax1.set_xticklabels(corr_matrix.columns, rotation=45, ha="right")
ax1.set_yticklabels(corr_matrix.columns)
ax1.set_title(
    "🔗 Matriz de Correlación\nEntre Métricas", fontsize=12, fontweight="bold"
)

# Agregar valores en la matriz
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        text = ax1.text(
            j,
            i,
            f"{corr_matrix.iloc[i, j]:.2f}",
            ha="center",
            va="center",
            color="white" if abs(corr_matrix.iloc[i, j]) > 0.5 else "black",
            fontweight="bold",
        )

# Colorbar para correlación
plt.colorbar(im1, ax=ax1, shrink=0.8)

# Gráfico 2: Tiempo vs Complejidad (Devoluciones)
ax2 = fig.add_subplot(gs[0, 1])
scatter1 = ax2.scatter(
    df_prs_merged["Devoluciones"],
    df_prs_merged["Tiempo_Resolucion"],
    c=df_prs_merged["Estandarizaciones_Codigo"],
    cmap="viridis",
    alpha=0.6,
    s=50,
    edgecolors="black",
    linewidth=0.3,
)
ax2.set_xlabel("Devoluciones")
ax2.set_ylabel("Tiempo de Resolución (días)")
ax2.set_title(
    "⏱️ Tiempo vs Complejidad\n(Color = Estandarizaciones)",
    fontsize=12,
    fontweight="bold",
)
plt.colorbar(scatter1, ax=ax2, shrink=0.8, label="Estandarizaciones")

# Agregar línea de tendencia
if len(df_prs_merged) > 1:
    z = np.polyfit(df_prs_merged["Devoluciones"], df_prs_merged["Tiempo_Resolucion"], 1)
    p = np.poly1d(z)
    ax2.plot(
        df_prs_merged["Devoluciones"],
        p(df_prs_merged["Devoluciones"]),
        "r--",
        alpha=0.8,
        linewidth=2,
    )

# Gráfico 3: Distribución de eficiencia por categorías
ax3 = fig.add_subplot(gs[0, 2])
categorias_count = [
    len(eficientes),
    len(necesitan_soporte),
    len(efficiency_relevant) - len(eficientes) - len(necesitan_soporte),
]
categorias_labels = ["Eficientes", "Necesitan\nSoporte", "Promedio"]
colors_cat = ["#2ecc71", "#e74c3c", "#f39c12"]

bars_cat = ax3.bar(categorias_labels, categorias_count, color=colors_cat, alpha=0.8)
ax3.set_title(
    "📊 Distribución por\nCategoría de Eficiencia", fontsize=12, fontweight="bold"
)
ax3.set_ylabel("Número de Desarrolladores")

# Agregar etiquetas en las barras
for i, bar in enumerate(bars_cat):
    height = bar.get_height()
    ax3.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.1,
        f"{int(height)}",
        ha="center",
        va="bottom",
        fontsize=12,
        fontweight="bold",
    )

# Gráfico 4: Top 10 más eficientes
ax4 = fig.add_subplot(gs[1, :])
if len(efficiency_relevant) > 0:
    # Calcular score combinado de eficiencia
    efficiency_relevant["Score_Eficiencia"] = (
        (1 / efficiency_relevant["Tiempo_Medio"])
        / (1 / efficiency_relevant["Tiempo_Medio"]).max()
    ) * 50 + (efficiency_relevant["Score_Consistencia"] / 100) * 50

    top_10_eff = efficiency_relevant.nlargest(10, "Score_Eficiencia")

    x_pos = range(len(top_10_eff))
    bars_eff = ax4.bar(
        x_pos,
        top_10_eff["Score_Eficiencia"],
        color=plt.cm.RdYlGn([x / 100 for x in top_10_eff["Score_Eficiencia"]]),
        alpha=0.8,
    )

    ax4.set_title(
        "🏆 TOP 10 DESARROLLADORES MÁS EFICIENTES (Velocidad + Consistencia)",
        fontsize=14,
        fontweight="bold",
    )
    ax4.set_ylabel("Score de Eficiencia (0-100)")
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(
        [name[:20] + "..." if len(name) > 20 else name for name in top_10_eff.index],
        rotation=45,
        ha="right",
    )

    # Agregar etiquetas con detalles
    for i, bar in enumerate(bars_eff):
        height = bar.get_height()
        author = top_10_eff.index[i]
        tiempo = top_10_eff.loc[author, "Tiempo_Medio"]
        ax4.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 1,
            f"{height:.1f}\n({tiempo:.1f}d)",
            ha="center",
            va="bottom",
            fontsize=9,
            fontweight="bold",
        )

# Gráfico 5: Análisis de outliers
ax5 = fig.add_subplot(gs[2, 0])
# Boxplot de tiempos de resolución
bp = ax5.boxplot(
    [df_prs_merged["Tiempo_Resolucion"]],
    patch_artist=True,
    boxprops=dict(facecolor="lightblue", alpha=0.7),
)
ax5.set_title(
    "📦 Detección de Outliers\nen Tiempo de Resolución", fontsize=12, fontweight="bold"
)
ax5.set_ylabel("Tiempo (días)")
ax5.set_xticklabels(["Todos los PRs"])

# Identificar outliers
Q1 = df_prs_merged["Tiempo_Resolucion"].quantile(0.25)
Q3 = df_prs_merged["Tiempo_Resolucion"].quantile(0.75)
IQR = Q3 - Q1
outliers = df_prs_merged[df_prs_merged["Tiempo_Resolucion"] > Q3 + 1.5 * IQR]

ax5.text(
    1.2,
    Q3 + 1.5 * IQR,
    f"Outliers: {len(outliers)}",
    fontsize=10,
    bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
)

# Gráfico 6: Consistencia vs Velocidad
ax6 = fig.add_subplot(gs[2, 1])
if len(efficiency_relevant) > 0:
    scatter2 = ax6.scatter(
        efficiency_relevant["Tiempo_Medio"],
        efficiency_relevant["Score_Consistencia"],
        c=efficiency_relevant["Total_PRs"],
        cmap="plasma",
        s=100,
        alpha=0.7,
        edgecolors="black",
        linewidth=0.5,
    )
    ax6.set_xlabel("Tiempo Promedio (días)")
    ax6.set_ylabel("Score de Consistencia")
    ax6.set_title(
        "🎯 Consistencia vs Velocidad\n(Tamaño = Total PRs)",
        fontsize=12,
        fontweight="bold",
    )

    # Agregar líneas de referencia
    ax6.axhline(
        y=efficiency_relevant["Score_Consistencia"].mean(),
        color="red",
        linestyle="--",
        alpha=0.7,
    )
    ax6.axvline(
        x=efficiency_relevant["Tiempo_Medio"].mean(),
        color="red",
        linestyle="--",
        alpha=0.7,
    )

    plt.colorbar(scatter2, ax=ax6, shrink=0.8, label="Total PRs")

# Gráfico 7: Evolución de calidad (estandarizaciones) en el tiempo
ax7 = fig.add_subplot(gs[2, 2])
if len(df_prs_merged) > 0:
    # Agrupar por mes y calcular promedio de estandarizaciones
    df_prs_merged["mes"] = df_prs_merged["Fecha_Creacion"].dt.to_period("M")
    quality_evolution = (
        df_prs_merged.groupby("mes")["Estandarizaciones_Codigo"].mean().tail(8)
    )

    ax7.plot(
        quality_evolution.index.astype(str),
        quality_evolution.values,
        "g-o",
        linewidth=3,
        markersize=8,
    )
    ax7.set_title(
        "📈 Evolución de Calidad\n(Estandarizaciones/PR)",
        fontsize=12,
        fontweight="bold",
    )
    ax7.set_ylabel("Promedio Estandarizaciones")
    ax7.set_xlabel("Mes")
    plt.setp(ax7.xaxis.get_majorticklabels(), rotation=45)

    # Agregar línea de tendencia
    if len(quality_evolution) > 1:
        z = np.polyfit(range(len(quality_evolution)), quality_evolution.values, 1)
        p = np.poly1d(z)
        ax7.plot(
            quality_evolution.index.astype(str),
            p(range(len(quality_evolution))),
            "r--",
            alpha=0.8,
            linewidth=2,
        )

# Tabla de desarrolladores que necesitan soporte
ax8 = fig.add_subplot(gs[3, :])
ax8.axis("tight")
ax8.axis("off")

if len(necesitan_soporte) > 0:
    # Preparar datos para la tabla
    support_data = []
    for author in necesitan_soporte.head(5).index:  # Top 5 que más necesitan soporte
        tiempo = necesitan_soporte.loc[author, "Tiempo_Medio"]
        consistencia = necesitan_soporte.loc[author, "Score_Consistencia"]
        total_prs = necesitan_soporte.loc[author, "Total_PRs"]

        # Determinar recomendación
        if tiempo > efficiency_relevant["Tiempo_Medio"].quantile(0.75):
            recomendacion = "🐌 Mejorar velocidad de desarrollo"
        elif consistencia < efficiency_relevant["Score_Consistencia"].quantile(0.25):
            recomendacion = "📊 Trabajar en consistencia"
        else:
            recomendacion = "🔄 Revisión general de proceso"

        support_data.append(
            [
                author[:25] + "..." if len(author) > 25 else author,
                f"{tiempo:.1f} días",
                f"{consistencia:.1f}%",
                f"{total_prs} PRs",
                recomendacion,
            ]
        )

    table_support = ax8.table(
        cellText=support_data,
        colLabels=[
            "Desarrollador",
            "Tiempo Promedio",
            "Consistencia",
            "Total PRs",
            "Recomendación",
        ],
        cellLoc="center",
        loc="center",
        colWidths=[0.25, 0.15, 0.15, 0.1, 0.35],
    )

    table_support.auto_set_font_size(False)
    table_support.set_fontsize(10)
    table_support.scale(1, 2)

    # Estilizar tabla
    for i in range(len(support_data) + 1):
        for j in range(5):
            cell = table_support[(i, j)]
            if i == 0:  # Header
                cell.set_facecolor("#e74c3c")
                cell.set_text_props(weight="bold", color="white")
            else:
                cell.set_facecolor("#fff3cd" if i % 2 == 0 else "#ffeaa7")

ax8.set_title(
    "⚠️ DESARROLLADORES QUE NECESITAN SOPORTE PRIORITARIO",
    fontsize=16,
    fontweight="bold",
    pad=20,
)

plt.suptitle(
    "🔍 ANÁLISIS DE EFICIENCIA Y CALIDAD DEL EQUIPO",
    fontsize=20,
    fontweight="bold",
    y=0.98,
)
plt.tight_layout()
plt.show()

# 4. Resumen ejecutivo de insights
print("\n\n💡 INSIGHTS Y RECOMENDACIONES:")
print("=" * 50)

# Correlación más fuerte
max_corr = corr_matrix.abs().unstack().sort_values(ascending=False)
max_corr = max_corr[max_corr < 1.0].iloc[0]
corr_vars = corr_matrix.abs().unstack().sort_values(ascending=False).index[1]

print(f"🔗 Correlación más fuerte: {corr_vars[0]} vs {corr_vars[1]} ({max_corr:.2f})")
print(f"⚡ Desarrolladores más eficientes: {len(eficientes)}")
print(f"⚠️  Desarrolladores que necesitan soporte: {len(necesitan_soporte)}")
print(
    f"📊 Tiempo promedio del equipo: {efficiency_relevant['Tiempo_Medio'].mean():.1f} días"
)
print(
    f"🎯 Coeficiente de variación promedio: {efficiency_relevant['Coef_Variacion_Tiempo'].mean():.1f}%"
)

if len(outliers) > 0:
    print(
        f"🚨 PRs outliers detectados: {len(outliers)} ({len(outliers) / len(df_prs_merged) * 100:.1f}% del total)"
    )

print("\n✅ Análisis de Eficiencia y Calidad completado!")

## 🔍 REPORTE 4: Métricas de Calidad del Código (Previas a QA/Revisión Senior)

Este reporte analiza específicamente las **devoluciones por PR**, proporcionando insights sobre la calidad inicial del código antes de pasar por controles de calidad y revisiones senior. Las devoluciones son un indicador clave de cuántos PRs no cumplen los estándares esperados en el primer intento.

In [ ]:
# REPORTE 4: MÉTRICAS DE CALIDAD DEL CÓDIGO (DEVOLUCIONES)
print("🔍 MÉTRICAS DE CALIDAD DEL CÓDIGO - ANÁLISIS DE DEVOLUCIONES")
print("=" * 65)

# 1. Estadísticas básicas de devoluciones
print("\n📊 ESTADÍSTICAS BÁSICAS DE DEVOLUCIONES:")
print("-" * 45)

devoluciones_stats = df_prs_merged["Devoluciones"].describe()
print(f"Total de PRs analizados: {len(df_prs_merged):,}")
print(f"Total de devoluciones: {df_prs_merged['Devoluciones'].sum():,}")
print(f"Promedio de devoluciones por PR: {devoluciones_stats['mean']:.2f}")
print(f"Mediana: {devoluciones_stats['50%']:.2f}")
print(f"Desviación estándar: {devoluciones_stats['std']:.2f}")
print(f"Mínimo: {int(devoluciones_stats['min'])}")
print(f"Máximo: {int(devoluciones_stats['max'])}")

# 2. Análisis de calidad inicial
prs_sin_devoluciones = len(df_prs_merged[df_prs_merged["Devoluciones"] == 0])
prs_con_devoluciones = len(df_prs_merged[df_prs_merged["Devoluciones"] > 0])
porcentaje_sin_devoluciones = (prs_sin_devoluciones / len(df_prs_merged)) * 100
porcentaje_con_devoluciones = (prs_con_devoluciones / len(df_prs_merged)) * 100

print(f"\n🎯 ANÁLISIS DE CALIDAD INICIAL:")
print("-" * 35)
print(
    f"PRs aprobados al primer intento: {prs_sin_devoluciones:,} ({porcentaje_sin_devoluciones:.1f}%)"
)
print(
    f"PRs que requirieron correcciones: {prs_con_devoluciones:,} ({porcentaje_con_devoluciones:.1f}%)"
)

# Interpretación de calidad
if porcentaje_sin_devoluciones >= 70:
    calidad_general = "EXCELENTE"
    color_calidad = "🟢"
elif porcentaje_sin_devoluciones >= 50:
    calidad_general = "BUENA"
    color_calidad = "🟡"
else:
    calidad_general = "REQUIERE MEJORA"
    color_calidad = "🔴"

print(f"Calidad inicial del código: {color_calidad} {calidad_general}")

# 3. Distribución de devoluciones
print(f"\n📈 DISTRIBUCIÓN DE DEVOLUCIONES:")
print("-" * 35)

# Crear categorías para las devoluciones
dev_bins = [-0.1, 0, 1, 3, 5, float("inf")]
dev_labels = [
    "Sin devoluciones",
    "Baja (1)",
    "Media (2-3)",
    "Alta (4-5)",
    "Muy Alta (>5)",
]
df_prs_merged["Devoluciones_Categoria"] = pd.cut(
    df_prs_merged["Devoluciones"], bins=dev_bins, labels=dev_labels, right=True
)

dev_distribution = df_prs_merged["Devoluciones_Categoria"].value_counts()
dev_percentages = (dev_distribution / len(df_prs_merged) * 100).round(1)

for categoria, count in dev_distribution.items():
    percentage = dev_percentages[categoria]
    print(f"{categoria}: {count:,} PRs ({percentage}%)")

print("\n✅ Estadísticas básicas de devoluciones completadas!")

In [ ]:
# 4. Análisis por desarrollador
print("\n👨‍💻 ANÁLISIS POR DESARROLLADOR:")
print("-" * 35)

# Estadísticas por autor
dev_por_autor = df_prs_merged.groupby('Autor')['Devoluciones'].agg([
    'count', 'sum', 'mean', 'std'
]).round(2)
dev_por_autor.columns = ['PRs_Total', 'Dev_Total', 'Dev_Promedio', 'Dev_Std']
dev_por_autor['Tasa_Exito'] = ((dev_por_autor['PRs_Total'] - df_prs_merged.groupby('Autor')['Devoluciones'].apply(lambda x: (x > 0).sum())) / dev_por_autor['PRs_Total'] * 100).round(1)
dev_por_autor = dev_por_autor.sort_values('Dev_Promedio', ascending=True)

print("🏆 TOP 10 DESARROLLADORES CON MEJOR CALIDAD INICIAL (Menos Devoluciones):")
mejores_devs = dev_por_autor[dev_por_autor['PRs_Total'] >= 3].head(10)
for autor, data in mejores_devs.iterrows():
    print(f"{autor}: {data['Dev_Promedio']:.1f} promedio, {data['Tasa_Exito']:.0f}% tasa de éxito")

print("\n⚠️  DESARROLLADORES QUE REQUIEREN ATENCIÓN (Más Devoluciones):")
devs_atencion = dev_por_autor[dev_por_autor['PRs_Total'] >= 3].tail(5)
for autor, data in devs_atencion.iterrows():
    print(f"{autor}: {data['Dev_Promedio']:.1f} promedio, {data['Tasa_Exito']:.0f}% tasa de éxito")

# 5. Análisis por repositorio
print("\n📦 ANÁLISIS POR REPOSITORIO:")
print("-" * 30)

dev_por_repo = df_prs_merged.groupby('Repositorio')['Devoluciones'].agg([
    'count', 'sum', 'mean', 'std'
]).round(2)
dev_por_repo.columns = ['PRs_Total', 'Dev_Total', 'Dev_Promedio', 'Dev_Std']
dev_por_repo['Tasa_Exito'] = ((dev_por_repo['PRs_Total'] - df_prs_merged.groupby('Repositorio')['Devoluciones'].apply(lambda x: (x > 0).sum())) / dev_por_repo['PRs_Total'] * 100).round(1)
dev_por_repo = dev_por_repo.sort_values('Dev_Promedio', ascending=True)

print("🏆 REPOSITORIOS CON MEJOR CALIDAD DE CÓDIGO:")
mejores_repos = dev_por_repo[dev_por_repo['PRs_Total'] >= 3].head(5)
for repo, data in mejores_repos.iterrows():
    print(f"{repo}: {data['Dev_Promedio']:.1f} promedio, {data['Tasa_Exito']:.0f}% tasa de éxito ({int(data['PRs_Total'])} PRs)")

print("⚠️  REPOSITORIOS QUE REQUIEREN ATENCIÓN:")
repos_atencion = dev_por_repo[dev_por_repo['PRs_Total'] >= 3].tail(3)
for repo, data in repos_atencion.iterrows():
    print(f"{repo}: {data['Dev_Promedio']:.1f} promedio, {data['Tasa_Exito']:.0f}% tasa de éxito ({int(data['PRs_Total'])} PRs)")

print("\n✅ Análisis por desarrollador y repositorio completado!")

In [ ]:
# 6. Crear visualizaciones específicas para devoluciones
plt.style.use('default')
fig = plt.figure(figsize=(20, 12))
gs = plt.GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# 1. Histograma principal de devoluciones (seaborn)
ax1 = fig.add_subplot(gs[0, :2])
import seaborn as sns
sns.histplot(data=df_prs_merged, x='Devoluciones', bins=15, kde=True, alpha=0.7, 
             color='skyblue', ax=ax1)
ax1.set_title('📊 Distribución de Devoluciones por PR\n(Histograma con Densidad)', 
             fontweight='bold', fontsize=14)
ax1.set_xlabel('Número de Devoluciones')
ax1.set_ylabel('Frecuencia')
ax1.axvline(df_prs_merged['Devoluciones'].mean(), color='red', 
           linestyle='--', linewidth=2, label=f'Promedio: {df_prs_merged["Devoluciones"].mean():.1f}')
ax1.axvline(df_prs_merged['Devoluciones'].median(), color='orange', 
           linestyle='--', linewidth=2, label=f'Mediana: {df_prs_merged["Devoluciones"].median():.1f}')
ax1.legend()

# Agregar anotaciones de insights
max_freq = ax1.patches[0].get_height()
for patch in ax1.patches:
    max_freq = max(max_freq, patch.get_height())

# Insight sobre cola larga
devoluciones_altas = len(df_prs_merged[df_prs_merged['Devoluciones'] > 3])
if devoluciones_altas > 0:
    ax1.text(0.7, 0.9, f'⚠️ Cola larga detectada:\n{devoluciones_altas} PRs con >3 devoluciones\n(Posibles problemas de calidad)', 
             transform=ax1.transAxes, bbox=dict(boxstyle="round,pad=0.5", facecolor="yellow", alpha=0.7),
             fontsize=10, verticalalignment='top')

# 2. Gráfico de barras por categorías
ax2 = fig.add_subplot(gs[0, 2])
categories_order = ['Sin devoluciones', 'Baja (1)', 'Media (2-3)', 'Alta (4-5)', 'Muy Alta (>5)']
valid_categories = [cat for cat in categories_order if cat in dev_distribution.index]
bars = ax2.bar(range(len(valid_categories)), 
               [dev_distribution[cat] for cat in valid_categories],
               color=['green', 'lightgreen', 'yellow', 'orange', 'red'][:len(valid_categories)])

ax2.set_xticks(range(len(valid_categories)))
ax2.set_xticklabels([cat.replace(' ', '\n') for cat in valid_categories], fontsize=9)
ax2.set_title('📈 PRs por Categoría\nde Devoluciones', fontweight='bold', fontsize=12)
ax2.set_ylabel('Número de PRs')

# Agregar valores en las barras
for bar, cat in zip(bars, valid_categories):
    height = bar.get_height()
    percentage = dev_percentages[cat]
    ax2.text(bar.get_x() + bar.get_width()/2, height + 0.5, 
            f'{int(height)}\n({percentage:.1f}%)', 
            ha='center', va='bottom', fontsize=9, fontweight='bold')

# 3. Top desarrolladores con mejor calidad
ax3 = fig.add_subplot(gs[1, 0])
top_devs = mejores_devs.head(8)
bars = ax3.barh(range(len(top_devs)), top_devs['Tasa_Exito'], 
                color='lightgreen', edgecolor='darkgreen')
ax3.set_yticks(range(len(top_devs)))
ax3.set_yticklabels([dev[:15] + '...' if len(dev) > 15 else dev for dev in top_devs.index])
ax3.set_title('🏆 Top Desarrolladores\n(Tasa de Éxito %)', fontweight='bold', fontsize=12)
ax3.set_xlabel('Tasa de Éxito (%)')

# Agregar valores en las barras
for i, (bar, value) in enumerate(zip(bars, top_devs['Tasa_Exito'])):
    ax3.text(value + 1, i, f'{value:.0f}%', va='center', fontsize=9)

# 4. Desarrolladores que requieren atención
ax4 = fig.add_subplot(gs[1, 1])
devs_problemas = devs_atencion
bars = ax4.barh(range(len(devs_problemas)), devs_problemas['Dev_Promedio'], 
                color='lightcoral', edgecolor='darkred')
ax4.set_yticks(range(len(devs_problemas)))
ax4.set_yticklabels([dev[:15] + '...' if len(dev) > 15 else dev for dev in devs_problemas.index])
ax4.set_title('⚠️ Desarrolladores que\nRequieren Atención', fontweight='bold', fontsize=12)
ax4.set_xlabel('Promedio de Devoluciones')

# Agregar valores en las barras
for i, (bar, value) in enumerate(zip(bars, devs_problemas['Dev_Promedio'])):
    ax4.text(value + 0.05, i, f'{value:.1f}', va='center', fontsize=9)

# 5. Análisis de repositorios
ax5 = fig.add_subplot(gs[1, 2])
repos_chart = mejores_repos
bars = ax5.bar(range(len(repos_chart)), repos_chart['Tasa_Exito'], 
               color='lightblue', edgecolor='darkblue')
ax5.set_xticks(range(len(repos_chart)))
ax5.set_xticklabels([repo[:8] + '...' if len(repo) > 8 else repo 
                    for repo in repos_chart.index], rotation=45, ha='right')
ax5.set_title('📦 Repositorios con\nMejor Calidad', fontweight='bold', fontsize=12)
ax5.set_ylabel('Tasa de Éxito (%)')

# Agregar valores en las barras
for bar, value in zip(bars, repos_chart['Tasa_Exito']):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
            f'{value:.0f}%', ha='center', va='bottom', fontsize=9)

# 6. Correlación devoluciones vs otras métricas
ax6 = fig.add_subplot(gs[2, 0])
if 'Tiempo_Resolucion' in df_prs_merged.columns:
    scatter = ax6.scatter(df_prs_merged['Devoluciones'], 
                         df_prs_merged['Tiempo_Resolucion'],
                         alpha=0.6, color='purple')
    ax6.set_title('🔗 Devoluciones vs\nTiempo de Resolución', fontweight='bold', fontsize=12)
    ax6.set_xlabel('Número de Devoluciones')
    ax6.set_ylabel('Tiempo de Resolución (días)')
    
    # Línea de tendencia
    z = np.polyfit(df_prs_merged['Devoluciones'], df_prs_merged['Tiempo_Resolucion'], 1)
    p = np.poly1d(z)
    ax6.plot(df_prs_merged['Devoluciones'], p(df_prs_merged['Devoluciones']), "r--", alpha=0.8)
    
    corr = df_prs_merged['Devoluciones'].corr(df_prs_merged['Tiempo_Resolucion'])
    ax6.text(0.05, 0.95, f'Correlación: {corr:.3f}', transform=ax6.transAxes, 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# 7. Correlación devoluciones vs estandarizaciones
ax7 = fig.add_subplot(gs[2, 1])
scatter = ax7.scatter(df_prs_merged['Devoluciones'], 
                     df_prs_merged['Estandarizaciones_Codigo'],
                     alpha=0.6, color='orange')
ax7.set_title('🔗 Devoluciones vs\nEstandarizaciones', fontweight='bold', fontsize=12)
ax7.set_xlabel('Número de Devoluciones')
ax7.set_ylabel('Estandarizaciones de Código')

# Línea de tendencia
z = np.polyfit(df_prs_merged['Devoluciones'], df_prs_merged['Estandarizaciones_Codigo'], 1)
p = np.poly1d(z)
ax7.plot(df_prs_merged['Devoluciones'], p(df_prs_merged['Devoluciones']), "r--", alpha=0.8)

corr = df_prs_merged['Devoluciones'].corr(df_prs_merged['Estandarizaciones_Codigo'])
ax7.text(0.05, 0.95, f'Correlación: {corr:.3f}', transform=ax7.transAxes, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

# 8. Box plot comparativo por repositorio
ax8 = fig.add_subplot(gs[2, 2])
repos_principales = df_prs_merged['Repositorio'].value_counts().head(4).index
box_data = [df_prs_merged[df_prs_merged['Repositorio'] == repo]['Devoluciones'].values 
           for repo in repos_principales]

bp = ax8.boxplot(box_data, labels=[repo[:8] + '...' if len(repo) > 8 else repo 
                                  for repo in repos_principales], patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax8.set_title('📦 Distribución de Devoluciones\npor Repositorio', fontweight='bold', fontsize=12)
ax8.set_ylabel('Número de Devoluciones')
ax8.tick_params(axis='x', rotation=45)

plt.suptitle('🔍 ANÁLISIS COMPLETO DE MÉTRICAS DE CALIDAD - DEVOLUCIONES', 
             fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n✅ Visualizaciones de métricas de calidad (devoluciones) completadas!")

In [ ]:
# 7. Insights específicos y recomendaciones
print("\n\n💡 INSIGHTS Y RECOMENDACIONES DE CALIDAD:")
print("=" * 50)

# Métricas clave
total_dev = df_prs_merged["Devoluciones"].sum()
promedio_dev = df_prs_merged["Devoluciones"].mean()
tasa_exito_general = porcentaje_sin_devoluciones

print(f"📊 MÉTRICAS CLAVE DE CALIDAD:")
print(f"   • Total de devoluciones: {total_dev:,}")
print(f"   • Promedio por PR: {promedio_dev:.2f}")
print(f"   • Tasa de éxito general: {tasa_exito_general:.1f}%")
print(f"   • Estado de calidad: {color_calidad} {calidad_general}")

# Análisis de cola larga
prs_cola_larga = len(df_prs_merged[df_prs_merged["Devoluciones"] > 3])
porcentaje_cola_larga = (prs_cola_larga / len(df_prs_merged)) * 100

print(f"\n🔍 ANÁLISIS DE COLA LARGA:")
if prs_cola_larga > 0:
    print(
        f"   • PRs con >3 devoluciones: {prs_cola_larga} ({porcentaje_cola_larga:.1f}%)"
    )
    if porcentaje_cola_larga > 15:
        print(f"   ⚠️  ALERTA: Cola larga significativa detectada")
        print(
            f"   📝 Sugiere: Problemas sistemáticos de calidad o criterios poco claros"
        )
    else:
        print(f"   ✅ Cola larga dentro de parámetros normales")
else:
    print(f"   ✅ No se detecta cola larga problemática")

# Correlaciones importantes
corr_tiempo = (
    df_prs_merged["Devoluciones"].corr(df_prs_merged["Tiempo_Resolucion"])
    if "Tiempo_Resolucion" in df_prs_merged.columns
    else 0
)
corr_est = df_prs_merged["Devoluciones"].corr(df_prs_merged["Estandarizaciones_Codigo"])

print(f"\n🔗 CORRELACIONES IMPORTANTES:")
if abs(corr_tiempo) > 0.3:
    print(f"   • Devoluciones vs Tiempo: {corr_tiempo:.3f} (correlación significativa)")
    print(f"     → Más devoluciones = Mayor tiempo de resolución")
else:
    print(f"   • Devoluciones vs Tiempo: {corr_tiempo:.3f} (correlación débil)")

if abs(corr_est) > 0.3:
    print(
        f"   • Devoluciones vs Estandarizaciones: {corr_est:.3f} (correlación significativa)"
    )
    if corr_est > 0:
        print(
            f"     → Más devoluciones = Más estandarizaciones aplicadas posteriormente"
        )
else:
    print(f"   • Devoluciones vs Estandarizaciones: {corr_est:.3f} (correlación débil)")

# Generar alertas específicas
alertas_calidad = []

if tasa_exito_general < 50:
    alertas_calidad.append(
        {
            "nivel": "CRÍTICA",
            "mensaje": f"Tasa de éxito muy baja ({tasa_exito_general:.1f}%)",
            "accion": "Revisar proceso completo de desarrollo y criterios de calidad",
        }
    )
elif tasa_exito_general < 70:
    alertas_calidad.append(
        {
            "nivel": "ALTA",
            "mensaje": f"Tasa de éxito baja ({tasa_exito_general:.1f}%)",
            "accion": "Implementar mejores prácticas de código y revisiones previas",
        }
    )

if porcentaje_cola_larga > 15:
    alertas_calidad.append(
        {
            "nivel": "ALTA",
            "mensaje": f"Cola larga detectada ({porcentaje_cola_larga:.1f}% con >3 devoluciones)",
            "accion": "Clarificar criterios de calidad y mejorar capacitación",
        }
    )

if promedio_dev > 2:
    alertas_calidad.append(
        {
            "nivel": "MEDIA",
            "mensaje": f"Promedio alto de devoluciones ({promedio_dev:.1f})",
            "accion": "Implementar checklist de calidad antes de PR",
        }
    )

# Mostrar alertas
if len(alertas_calidad) > 0:
    print(f"\n🚨 ALERTAS DE CALIDAD:")
    for i, alerta in enumerate(alertas_calidad, 1):
        icono = (
            "🔴"
            if alerta["nivel"] == "CRÍTICA"
            else "🟠"
            if alerta["nivel"] == "ALTA"
            else "🟡"
        )
        print(f"   {i}. {icono} {alerta['nivel']}: {alerta['mensaje']}")
        print(f"      → Acción: {alerta['accion']}")
else:
    print(f"\n✅ No se detectaron alertas críticas de calidad")

# Mejores prácticas identificadas
print(f"\n🏆 MEJORES PRÁCTICAS IDENTIFICADAS:")
if len(mejores_devs) > 0:
    mejor_dev = mejores_devs.index[0]
    mejor_tasa = mejores_devs.iloc[0]["Tasa_Exito"]
    print(f"   • Mejor desarrollador: {mejor_dev} ({mejor_tasa:.0f}% tasa de éxito)")

if len(mejores_repos) > 0:
    mejor_repo = mejores_repos.index[0]
    mejor_repo_tasa = mejores_repos.iloc[0]["Tasa_Exito"]
    print(
        f"   • Mejor repositorio: {mejor_repo} ({mejor_repo_tasa:.0f}% tasa de éxito)"
    )

# Recomendaciones prioritarias
print(f"\n🎯 RECOMENDACIONES PRIORITARIAS:")

if tasa_exito_general < 70:
    print(f"   1. 🔧 Implementar código de revisión peer-to-peer antes de PR")
    print(f"   2. 📚 Crear guías de estándares de código más claras")
    print(f"   3. 🎓 Programa de capacitación en mejores prácticas")

if len(devs_atencion) > 0:
    print(f"   4. 👥 Mentoring específico para desarrolladores con baja tasa de éxito")
    print(f"   5. 🔍 Revisión de criterios de calidad para uniformidad")

if porcentaje_cola_larga > 10:
    print(f"   6. 📋 Checklist obligatorio de calidad antes de enviar PR")
    print(f"   7. 🤖 Herramientas automatizadas de análisis de código")

print(f"\n📋 RESUMEN EJECUTIVO:")
print(f"   • Calidad inicial: {color_calidad} {calidad_general}")
print(f"   • Tasa de éxito: {tasa_exito_general:.1f}%")
print(
    f"   • Desarrolladores destacados: {len(mejores_devs[mejores_devs['Tasa_Exito'] > 80])}"
)
print(
    f"   • Repositorios con buena calidad: {len(mejores_repos[mejores_repos['Tasa_Exito'] > 80])}"
)

# Interpretación de cola larga específica
if prs_cola_larga > 0:
    print(f"   • Cola larga: {prs_cola_larga} PRs requieren atención especial")
    if porcentaje_cola_larga > 15:
        print(f"   🔴 Indica problemas sistemáticos de calidad")
    else:
        print(f"   🟡 Dentro de parámetros normales pero monitoreable")

print("\n✅ Análisis de Métricas de Calidad del Código COMPLETADO!")
print("=" * 55)

## 👥 REPORTE 5: Análisis de Devoluciones por Autor

Este reporte se enfoca específicamente en el análisis de **devoluciones por desarrollador**, identificando patrones de calidad individual y proporcionando insights sobre qué desarrolladores entregan código que cumple los criterios de QA versus aquellos que requieren múltiples iteraciones.

In [ ]:
# REPORTE 5: ANÁLISIS DE DEVOLUCIONES POR AUTOR
print("👥 ANÁLISIS DETALLADO DE DEVOLUCIONES POR AUTOR")
print("=" * 55)

# 1. Estadísticas completas por autor
print("\n📊 ESTADÍSTICAS DETALLADAS POR DESARROLLADOR:")
print("-" * 50)

# Crear análisis completo por autor
autor_stats = (
    df_prs_merged.groupby("Autor")["Devoluciones"]
    .agg(
        [
            "count",  # Total de PRs
            "sum",  # Total de devoluciones
            "mean",  # Promedio de devoluciones
            "median",  # Mediana de devoluciones
            "std",  # Desviación estándar
            "min",  # Mínimo de devoluciones
            "max",  # Máximo de devoluciones
        ]
    )
    .round(2)
)

autor_stats.columns = [
    "PRs_Total",
    "Dev_Total",
    "Dev_Promedio",
    "Dev_Mediana",
    "Dev_Std",
    "Dev_Min",
    "Dev_Max",
]

# Calcular métricas adicionales
autor_stats["Tasa_Exito"] = (
    (
        autor_stats["PRs_Total"]
        - df_prs_merged.groupby("Autor")["Devoluciones"].apply(lambda x: (x > 0).sum())
    )
    / autor_stats["PRs_Total"]
    * 100
).round(1)
autor_stats["Coef_Variacion"] = (
    (autor_stats["Dev_Std"] / autor_stats["Dev_Promedio"] * 100).fillna(0).round(1)
)
autor_stats["PRs_Sin_Dev"] = df_prs_merged.groupby("Autor")["Devoluciones"].apply(
    lambda x: (x == 0).sum()
)
autor_stats["PRs_Con_Dev"] = autor_stats["PRs_Total"] - autor_stats["PRs_Sin_Dev"]

# Filtrar autores con al menos 3 PRs para análisis más representativo
autor_stats_filtered = autor_stats[autor_stats["PRs_Total"] >= 3].copy()

print(f"Total de desarrolladores analizados: {len(autor_stats)}")
print(f"Desarrolladores con ≥3 PRs: {len(autor_stats_filtered)}")
print(f"Promedio general de devoluciones: {df_prs_merged['Devoluciones'].mean():.2f}")

# 2. Clasificación de desarrolladores por calidad
print(f"\n🏆 CLASIFICACIÓN POR CALIDAD DE CÓDIGO:")
print("-" * 40)

# Clasificar desarrolladores
excelentes = autor_stats_filtered[
    (autor_stats_filtered["Tasa_Exito"] >= 80)
    & (autor_stats_filtered["Dev_Promedio"] <= 1)
]
buenos = autor_stats_filtered[
    (autor_stats_filtered["Tasa_Exito"] >= 60)
    & (autor_stats_filtered["Dev_Promedio"] <= 2)
]
regulares = autor_stats_filtered[
    (autor_stats_filtered["Tasa_Exito"] >= 40)
    & (autor_stats_filtered["Dev_Promedio"] <= 3)
]
necesitan_atencion = autor_stats_filtered[
    (autor_stats_filtered["Tasa_Exito"] < 40)
    | (autor_stats_filtered["Dev_Promedio"] > 3)
]

print(f"🟢 EXCELENTES (Tasa ≥80%, Prom ≤1): {len(excelentes)} desarrolladores")
for dev in excelentes.head(3).index:
    data = excelentes.loc[dev]
    print(
        f"   • {dev}: {data['Tasa_Exito']:.0f}% éxito, {data['Dev_Promedio']:.1f} prom ({int(data['PRs_Total'])} PRs)"
    )

print(f"\n🟡 BUENOS (Tasa ≥60%, Prom ≤2): {len(buenos)} desarrolladores")
for dev in buenos.head(3).index:
    data = buenos.loc[dev]
    print(
        f"   • {dev}: {data['Tasa_Exito']:.0f}% éxito, {data['Dev_Promedio']:.1f} prom ({int(data['PRs_Total'])} PRs)"
    )

print(f"\n🟠 REGULARES: {len(regulares)} desarrolladores")
for dev in regulares.head(3).index:
    data = regulares.loc[dev]
    print(
        f"   • {dev}: {data['Tasa_Exito']:.0f}% éxito, {data['Dev_Promedio']:.1f} prom ({int(data['PRs_Total'])} PRs)"
    )

print(f"\n🔴 NECESITAN ATENCIÓN: {len(necesitan_atencion)} desarrolladores")
for dev in necesitan_atencion.index:
    data = necesitan_atencion.loc[dev]
    print(
        f"   • {dev}: {data['Tasa_Exito']:.0f}% éxito, {data['Dev_Promedio']:.1f} prom ({int(data['PRs_Total'])} PRs)"
    )

print("\n✅ Análisis estadístico por autor completado!")

In [ ]:
# 3. Crear visualizaciones específicas por autor
import seaborn as sns

plt.style.use("default")
fig = plt.figure(figsize=(20, 16))
gs = plt.GridSpec(4, 2, figure=fig, hspace=0.4, wspace=0.3)

# 1. Gráfico de barras principal: Promedio de devoluciones por autor (seaborn)
ax1 = fig.add_subplot(gs[0, :])
# Ordenar por promedio de devoluciones para mejor visualización
autor_stats_sorted = autor_stats_filtered.sort_values("Dev_Promedio")

# Crear colores basados en la clasificación
colors_clasificacion = []
for autor in autor_stats_sorted.index:
    if autor in excelentes.index:
        colors_clasificacion.append("green")
    elif autor in buenos.index:
        colors_clasificacion.append("gold")
    elif autor in regulares.index:
        colors_clasificacion.append("orange")
    else:
        colors_clasificacion.append("red")

bars = sns.barplot(
    data=df_prs_merged[df_prs_merged["Autor"].isin(autor_stats_sorted.index)],
    x="Autor",
    y="Devoluciones",
    estimator=np.mean,
    order=autor_stats_sorted.index,
    palette=colors_clasificacion,
    ax=ax1,
)
ax1.set_title(
    "📊 PROMEDIO DE DEVOLUCIONES POR DESARROLLADOR\n(Verde=Excelente, Amarillo=Bueno, Naranja=Regular, Rojo=Necesita Atención)",
    fontweight="bold",
    fontsize=14,
)
ax1.set_xlabel("Desarrollador")
ax1.set_ylabel("Promedio de Devoluciones")
ax1.tick_params(axis="x", rotation=45)

# Agregar línea de promedio general
promedio_general = df_prs_merged["Devoluciones"].mean()
ax1.axhline(
    y=promedio_general,
    color="red",
    linestyle="--",
    alpha=0.7,
    label=f"Promedio General: {promedio_general:.2f}",
)
ax1.legend()

# Agregar valores en las barras
for i, (bar, autor) in enumerate(zip(bars.patches, autor_stats_sorted.index)):
    valor = autor_stats_sorted.loc[autor, "Dev_Promedio"]
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.05,
        f"{valor:.1f}",
        ha="center",
        va="bottom",
        fontsize=9,
        fontweight="bold",
    )

# 2. Box plot: Distribución de devoluciones por autor (seaborn)
ax2 = fig.add_subplot(gs[1, :])
# Limitar a desarrolladores más activos para mejor visualización
top_devs_boxplot = (
    autor_stats_filtered.sort_values("PRs_Total", ascending=False).head(8).index
)
df_boxplot = df_prs_merged[df_prs_merged["Autor"].isin(top_devs_boxplot)]

sns.boxplot(data=df_boxplot, x="Autor", y="Devoluciones", ax=ax2)
ax2.set_title(
    "📦 DISTRIBUCIÓN DE DEVOLUCIONES POR DESARROLLADOR (Top 8 más activos)",
    fontweight="bold",
    fontsize=14,
)
ax2.set_xlabel("Desarrollador")
ax2.set_ylabel("Número de Devoluciones")
ax2.tick_params(axis="x", rotation=45)

# 3. Gráfico de dispersión: PRs vs Promedio de devoluciones
ax3 = fig.add_subplot(gs[2, 0])
scatter = ax3.scatter(
    autor_stats_filtered["PRs_Total"],
    autor_stats_filtered["Dev_Promedio"],
    c=[
        colors_clasificacion[list(autor_stats_sorted.index).index(autor)]
        if autor in autor_stats_sorted.index
        else "gray"
        for autor in autor_stats_filtered.index
    ],
    s=100,
    alpha=0.7,
    edgecolors="black",
)

ax3.set_title(
    "🎯 Experiencia vs Calidad\n(PRs Total vs Promedio Devoluciones)",
    fontweight="bold",
    fontsize=12,
)
ax3.set_xlabel("Total de PRs")
ax3.set_ylabel("Promedio de Devoluciones")

# Agregar nombres de desarrolladores problemáticos
for autor in necesitan_atencion.index:
    if autor in autor_stats_filtered.index:
        x = autor_stats_filtered.loc[autor, "PRs_Total"]
        y = autor_stats_filtered.loc[autor, "Dev_Promedio"]
        ax3.annotate(
            autor[:10],
            (x, y),
            xytext=(5, 5),
            textcoords="offset points",
            fontsize=8,
            alpha=0.8,
        )

# 4. Gráfico de barras: Tasa de éxito por desarrollador
ax4 = fig.add_subplot(gs[2, 1])
autor_stats_exito = autor_stats_filtered.sort_values("Tasa_Exito", ascending=False)
bars_exito = ax4.bar(
    range(len(autor_stats_exito)),
    autor_stats_exito["Tasa_Exito"],
    color=[
        colors_clasificacion[list(autor_stats_sorted.index).index(autor)]
        if autor in autor_stats_sorted.index
        else "gray"
        for autor in autor_stats_exito.index
    ],
)

ax4.set_xticks(range(len(autor_stats_exito)))
ax4.set_xticklabels(
    [
        autor[:10] + "..." if len(autor) > 10 else autor
        for autor in autor_stats_exito.index
    ],
    rotation=45,
    ha="right",
)
ax4.set_title(
    "🏆 Tasa de Éxito por Desarrollador\n(% PRs sin devoluciones)",
    fontweight="bold",
    fontsize=12,
)
ax4.set_ylabel("Tasa de Éxito (%)")
ax4.axhline(y=70, color="orange", linestyle="--", alpha=0.7, label="Meta: 70%")
ax4.legend()

# 5. Heatmap de métricas por desarrollador
ax5 = fig.add_subplot(gs[3, :])
# Seleccionar métricas clave para el heatmap
metrics_heatmap = autor_stats_filtered[
    ["Dev_Promedio", "Tasa_Exito", "Coef_Variacion", "PRs_Total"]
].copy()
metrics_heatmap_norm = metrics_heatmap.copy()

# Normalizar para el heatmap (invertir donde menor es mejor)
metrics_heatmap_norm["Dev_Promedio"] = 100 - (
    metrics_heatmap["Dev_Promedio"] / metrics_heatmap["Dev_Promedio"].max() * 100
)
metrics_heatmap_norm["Coef_Variacion"] = 100 - (
    metrics_heatmap["Coef_Variacion"] / metrics_heatmap["Coef_Variacion"].max() * 100
)
metrics_heatmap_norm["PRs_Total"] = (
    metrics_heatmap["PRs_Total"] / metrics_heatmap["PRs_Total"].max() * 100
)

# Limitar a top 10 desarrolladores por PRs para mejor visualización
top_10_devs = metrics_heatmap_norm.sort_values("PRs_Total", ascending=False).head(10)

im = ax5.imshow(top_10_devs.T.values, cmap="RdYlGn", aspect="auto", vmin=0, vmax=100)
ax5.set_xticks(range(len(top_10_devs)))
ax5.set_xticklabels(
    [dev[:12] + "..." if len(dev) > 12 else dev for dev in top_10_devs.index],
    rotation=45,
    ha="right",
)
ax5.set_yticks(range(len(top_10_devs.columns)))
ax5.set_yticklabels(
    ["Calidad Código", "Tasa Éxito (%)", "Consistencia", "Experiencia (PRs)"]
)
ax5.set_title(
    "🔥 MAPA DE CALOR: MÉTRICAS DE DESARROLLADORES\n(Verde=Excelente, Amarillo=Bueno, Rojo=Necesita Mejora)",
    fontweight="bold",
    fontsize=14,
)

# Agregar valores en el heatmap
for i in range(len(top_10_devs.columns)):
    for j in range(len(top_10_devs)):
        value = top_10_devs.iloc[j, i]
        ax5.text(
            j,
            i,
            f"{value:.0f}",
            ha="center",
            va="center",
            fontweight="bold",
            fontsize=9,
            color="white" if value < 50 else "black",
        )

# Agregar colorbar
cbar = plt.colorbar(im, ax=ax5, shrink=0.8)
cbar.set_label("Puntuación de Calidad (0-100)")

plt.suptitle(
    "👥 ANÁLISIS COMPLETO DE DEVOLUCIONES POR DESARROLLADOR",
    fontsize=18,
    fontweight="bold",
    y=0.98,
)
plt.tight_layout()
plt.show()

print("\n✅ Visualizaciones de análisis por autor completadas!")

In [ ]:
# 4. Insights específicos y planes de acción por desarrollador
print("\n\n💡 INSIGHTS DETALLADOS Y PLANES DE ACCIÓN:")
print("=" * 55)

# Análisis de patrones por categoría
print(f"📊 RESUMEN POR CATEGORÍAS:")
print(
    f"   🟢 Desarrolladores EXCELENTES: {len(excelentes)} ({len(excelentes) / len(autor_stats_filtered) * 100:.1f}%)"
)
print(
    f"   🟡 Desarrolladores BUENOS: {len(buenos)} ({len(buenos) / len(autor_stats_filtered) * 100:.1f}%)"
)
print(
    f"   🟠 Desarrolladores REGULARES: {len(regulares)} ({len(regulares) / len(autor_stats_filtered) * 100:.1f}%)"
)
print(
    f"   🔴 Desarrolladores que NECESITAN ATENCIÓN: {len(necesitan_atencion)} ({len(necesitan_atencion) / len(autor_stats_filtered) * 100:.1f}%)"
)

# Análisis de problemáticas específicas
print(f"\n🔍 ANÁLISIS DE PROBLEMÁTICAS ESPECÍFICAS:")

# Desarrolladores con alta variabilidad
alta_variabilidad = autor_stats_filtered[autor_stats_filtered["Coef_Variacion"] > 100]
if len(alta_variabilidad) > 0:
    print(f"\n⚠️  DESARROLLADORES CON ALTA VARIABILIDAD ({len(alta_variabilidad)}):")
    print("   → Posible falta de comprensión consistente de requisitos")
    for dev in alta_variabilidad.index:
        cv = alta_variabilidad.loc[dev, "Coef_Variacion"]
        prom = alta_variabilidad.loc[dev, "Dev_Promedio"]
        print(f"   • {dev}: CV={cv:.0f}%, Promedio={prom:.1f}")

# Desarrolladores con muchos PRs pero alta tasa de devoluciones
experimentados_problematicos = autor_stats_filtered[
    (autor_stats_filtered["PRs_Total"] >= 5)
    & (autor_stats_filtered["Dev_Promedio"] > 2)
]
if len(experimentados_problematicos) > 0:
    print(
        f"\n⚠️  DESARROLLADORES EXPERIMENTADOS CON PROBLEMAS DE CALIDAD ({len(experimentados_problematicos)}):"
    )
    print(
        "   → Posibles malos hábitos establecidos o falta de actualización en estándares"
    )
    for dev in experimentados_problematicos.index:
        prs = experimentados_problematicos.loc[dev, "PRs_Total"]
        prom = experimentados_problematicos.loc[dev, "Dev_Promedio"]
        tasa = experimentados_problematicos.loc[dev, "Tasa_Exito"]
        print(f"   • {dev}: {int(prs)} PRs, {prom:.1f} prom dev, {tasa:.0f}% éxito")

# Desarrolladores nuevos con potencial
nuevos_prometedores = autor_stats_filtered[
    (autor_stats_filtered["PRs_Total"] <= 5)
    & (autor_stats_filtered["Dev_Promedio"] <= 1.5)
]
if len(nuevos_prometedores) > 0:
    print(f"\n🌟 DESARROLLADORES NUEVOS CON POTENCIAL ({len(nuevos_prometedores)}):")
    print("   → Mantener buenas prácticas y mentoring")
    for dev in nuevos_prometedores.index:
        prs = nuevos_prometedores.loc[dev, "PRs_Total"]
        prom = nuevos_prometedores.loc[dev, "Dev_Promedio"]
        print(f"   • {dev}: {int(prs)} PRs, {prom:.1f} prom dev")

# Generar planes de acción específicos
print(f"\n🎯 PLANES DE ACCIÓN ESPECÍFICOS:")

# Para desarrolladores que necesitan atención
if len(necesitan_atencion) > 0:
    print(f"\n🔴 PLAN PARA DESARROLLADORES QUE NECESITAN ATENCIÓN:")
    for dev in necesitan_atencion.index:
        data = necesitan_atencion.loc[dev]
        print(f"\n   👤 {dev}:")
        print(
            f"      📊 Métricas: {data['Tasa_Exito']:.0f}% éxito, {data['Dev_Promedio']:.1f} prom dev"
        )

        # Diagnóstico específico
        if data["Dev_Promedio"] > 3:
            print(f"      🔍 Diagnóstico: Promedio muy alto de devoluciones")
            print(f"      💡 Recomendaciones:")
            print(f"         - Revisión exhaustiva de estándares de código")
            print(f"         - Mentoring 1:1 con desarrollador senior")
            print(f"         - Implementar checklist personal antes de PR")

        if data["Tasa_Exito"] < 30:
            print(f"      🔍 Diagnóstico: Muy baja tasa de éxito al primer intento")
            print(f"      💡 Recomendaciones:")
            print(f"         - Capacitación específica en testing")
            print(f"         - Revisión de comprensión de requisitos")
            print(f"         - Pair programming con desarrolladores excelentes")

        if data["Coef_Variacion"] > 150:
            print(f"      🔍 Diagnóstico: Alta inconsistencia en calidad")
            print(f"      💡 Recomendaciones:")
            print(f"         - Estandarizar proceso personal de desarrollo")
            print(f"         - Capacitación en metodologías de calidad")

# Para desarrolladores excelentes
if len(excelentes) > 0:
    print(f"\n🟢 PLAN PARA DESARROLLADORES EXCELENTES:")
    print(f"   🎯 Acciones:")
    print(f"      - Asignar como mentores de desarrolladores que necesitan atención")
    print(f"      - Documentar sus mejores prácticas")
    print(f"      - Considerar para roles de liderazgo técnico")
    print(f"      - Asignar PRs más complejos o críticos")

# Métricas de seguimiento recomendadas
print(f"\n📈 MÉTRICAS DE SEGUIMIENTO RECOMENDADAS:")
print(f"   📊 Semanales:")
print(f"      • Tasa de éxito por desarrollador")
print(f"      • Promedio de devoluciones en PRs nuevos")
print(f"   📊 Mensuales:")
print(f"      • Evolución de coeficiente de variación")
print(f"      • Comparación con benchmarks del equipo")
print(f"   📊 Trimestrales:")
print(f"      • Reclasificación de desarrolladores")
print(f"      • Evaluación de efectividad de planes de mejora")

# Impacto en el equipo
promedio_team = autor_stats_filtered["Dev_Promedio"].mean()
desarrolladores_sobre_promedio = len(
    autor_stats_filtered[autor_stats_filtered["Dev_Promedio"] > promedio_team]
)
impacto_mejora = desarrolladores_sobre_promedio / len(autor_stats_filtered) * 100

print(f"\n💰 IMPACTO POTENCIAL:")
print(
    f"   • {desarrolladores_sobre_promedio} desarrolladores están sobre el promedio del equipo"
)
print(
    f"   • Mejorando estos desarrolladores se podría reducir {impacto_mejora:.1f}% las devoluciones"
)
print(
    f"   • Tiempo estimado ahorrado: ~{impacto_mejora * 0.5:.1f} horas/semana en re-trabajo"
)

print("\n✅ Análisis de Devoluciones por Autor COMPLETADO!")
print("=" * 55)